<h1>autobuildicdb NPD</h1><br>
Auto-populate a blank SQL Server IC database with data from the <a href="https://factpages.npd.no/factpages/Default.aspx?culture=en", target="_blank">NPD FactPages</a>.

<b>Part 1. Downloads the following data types, reformats for IC and exports to .csv</b>

Exploration well headers<br>
Development well headers<br>
Cores<br>
Core photos (also saves .jpf files to disc)<br>
Thin sections<br>
CO2<br>
Oil samples<br>
Lithostratigraphy<br>
Drill stem tests<br>
Casing and leak-off tests<br>
Drilling mud<br>
Documents<br>

<b>Part 2. Connects to the database and populates tables</b><br>

dbo.WELLS<br>
dbo.t_WellsUserFields<br>
dbo.t_WellsUserFieldsValues<br>
dbo.PROJECTS (creates 4 dynamic projects)<br>
dbo.WELLQUERIES (creates one query per project)<br>
dbo.PROJECTWELLS<br>
dbo.tablenames<br>
dbo.intervalcolumns<br>
dbo.DATA_Lithostrat<br>
dbo.DATA_Core

Only handles data for Well Headers, References and Lithostrat just now.<br>
Creates dynamic IC projects, well queries, and builds text dictionaries from Lithostrat.<br>

<b>Part 3. Download additional data</b>

Download all Core Photos to file (using hyperlinks).<br>
Download and unzip shapefiles from 'NPD Map Services'.<br>

In [1]:
# Uncomment your chosen data source -
    # web: downloads data live from NPD using permanent links
    # file: if you have manually downloaded data in Excel format and saved to 'input data' folder 
        # Use file for for testing purposes

data_source = 'web'
#data_source = 'file'

In [ ]:
# # Select which NPD data types to work with

# data_to_load = [exploration_well_headers
#                 ,development_well_headers
#                 ,cores
#                 ,core_photos
#                 ,thin_sections
#                 ,co2
#                 ,oil_samples
#                 ,lithostratigraphy
#                 ,drill_stem_tests
#                 ,casing_and_leak-off_tests
#                 ,drilling mud
#                 ,documents
#                ]

In [2]:
# Writing to C:\Alan Python\autobuildicdb

folder = 'output data'

def output_to_csv(file, df):

    filepath = '{}/{}.csv'.format(folder, file)
    print('Saved to:', filepath)
    
    df.to_csv(filepath, index=False, encoding='utf-8-sig')

    return pd.read_csv(filepath).head(3)

<h2>Part 1. Download NPD data, reformat for IC and output to .csv</h2>
    
<h3>Well Headers</h3>

In [3]:
import pandas as pd
import numpy as np
from pandas import ExcelFile
from pandas import ExcelWriter
import urllib.request
import requests, zipfile, io

#Change Pandas display settings to show all columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
#pd.set_option('display.max_rows', 500)

In [4]:
# Download the latest NPD well headers in Excel format
# Navigate to NPD Factpages > Wellbore > Table View > Exploration/Development > All - Long List> Export Excel.
# Assign to two dataframes, one for Exploraion wells and one for Development wells


if data_source == 'web':
    df_explo = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_exploration_all&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.169&CultureCode=en')
    df_dev = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_development_all&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.169&CultureCode=en')

if data_source == 'file':
    # Navigate to NPD Factpages > Wellbore > Table View > Exploration/Development > All - Long List> Export Excel.
    df_explo = pd.read_excel('input data/wellbore_exploration_all.xlsx')
    df_dev = pd.read_excel('input data/wellbore_development_all.xlsx')

# Print the original column titles in each dataframe.
print("\nExploration well header column titles:")
print(list(df_explo.columns))
print("\nDevelopment well header column titles:")
print(list(df_dev.columns))


Exploration well header column titles:
['Wellbore name', 'Well name', 'Drilling operator', 'Drilled in production licence', 'Purpose', 'Status', 'Content', 'Type', 'Subsea', 'Entered date', 'Completed date', 'Field', 'Drill permit', 'Discovery', 'Discovery wellbore', 'Bottom hole temperature [°C]', 'Sitesurvey', 'Seismic location', 'Maximum inclination [°]', 'Kelly bushing elevation [m]', 'Final vertical depth (TVD) [m RKB]', 'Total depth (MD) [m RKB]', 'Water depth [m]', 'Kick off  point [m RKB]', 'Oldest penetrated age', 'Oldest penetrated formation', 'Main area', 'Drilling facility', 'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 'Multilateral', 'Purpose - planned', 'Entry year', 'Completed year', 'Reclassified from/to wellbore', 'Reentry activity', 'Plot symbol', '1st level with HC, formation', '1st level with HC, age', '2nd level with HC, formation', '2nd level with HC, age', '3rd level with HC, formation', '3rd level with HC, age', 'Dril

In [5]:
(num_explo_rows, num_explo_cols) = df_explo.shape
(num_dev_rows, num_dev_cols) = df_dev.shape
print('{} rows and {} columns in Exploration wells.'.format(num_explo_rows, num_explo_cols))
print('{} rows and {} columns in Development wells.'.format(num_dev_rows, num_dev_cols))

1933 rows and 87 columns in Exploration wells.
5154 rows and 75 columns in Development wells.


In [6]:
print('The first 3 rows of Exploration wells:')
df_explo.head(3)

The first 3 rows of Exploration wells:


,Wellbore name,Well name,Drilling operator,Drilled in production licence,Purpose,Status,Content,Type,Subsea,Entered date,Completed date,Field,Drill permit,Discovery,Discovery wellbore,Bottom hole temperature [°C],Sitesurvey,Seismic location,Maximum inclination [°],Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Main area,Drilling facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Purpose - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,Plot symbol,"1st level with HC, formation","1st level with HC, age","2nd level with HC, formation","2nd level with HC, age","3rd level with HC, formation","3rd level with HC, age",Drilling days,Reentry,Prod. licence for drilling target,Plugged and abondon date,Plugged date,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",Pressrelease url,FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,Reclassified date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,NPDID site survey,Date main level updated,Date all updated,Date sync NPD
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,1989-03-20,1989-06-04,BLANE,604-L,1/2-1 Blane,YES,147.0,None,PW 8303A - 10 SP. 290,2.0,22.0,NaN,3574.0,72.0,NaN,CAMPANIAN,TOR FM,NORTH SEA,ROSS ISLE,SEMISUB STEEL,MOVEABLE,12,NO,WILDCAT,1989,1989,None,None,5,FORTIES FM,PALEOCENE,NaN,NaN,,,77,NO,NaN,NaT,NaT,ED50,56,53,15.07,N,2,28,35.70,E,56.887519,2.476583,6305128.26,468106.29,31,1,2,,1,NaN,NaN,,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382,initial,,2007-12-19,1991-06-04,NaT,1382,43814.0,3437650.0,296245.0,0,21956.0,NaN,2019-10-03,2019-10-03,16.02.2020
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P&A,OIL SHOWS,EXPLORATION,NO,2005-12-14,2006-02-02,None,1103-L,None,NO,138.0,None,inline 7429-trace 4824 Survey PGS CGMNOR,4.9,40.0,3432.0,3434.0,74.0,NaN,PALEOCENE,EKOFISK FM,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,12,NO,WILDCAT,2005,2006,None,None,12,NaN,NaN,NaN,NaN,,,51,NO,NaN,NaT,NaT,ED50,56,59,32.00,N,2,29,47.66,E,56.992222,2.496572,6316774.33,469410.10,31,1,2,,2,NaN,NaN,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2006/1-2-2/,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=5192,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=5192,initial,,2008-08-15,2008-02-02,NaT,5192,NaN,NaN,278245.0,0,2424919.0,NaN,2019-10-03,2019-10-03,16.02.2020
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P&A,GAS,EXPLORATION,NO,1968-07-06,1968-11-11,None,15-L,1/3-1,YES,182.0,None,LINE 5651 SP. E165,18.0,26.0,NaN,4877.0,71.0,NaN,LATE PERMIAN,ZECHSTEIN GP,NORTH SEA,ORION,JACK-UP 3 LEGS,MOVEABLE,1-A,NO,WILDCAT,1968,1968,None,None,9,TOR FM,LATE CRETACEOUS,CROMER KNOLL GP,EARLY CRETACEOUS,,,129,NO,NaN,NaT,NaT,ED50,56,51,21.00,N,2,51,5.00,E,56.855833,2.851389,6301488.86,490936.87,31,1,3,,1,NaN,NaN,,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=154,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=154,initial,,2010-04-30,1970-11-11,NaT,154,43820.0,NaN,288604.0,0,20844.0,NaN,2019-10-03,2019-10-03,16.02.2020


In [7]:
print('The first 3 rows of Development wells:')
df_dev.head(3)

The first 3 rows of Development wells:


,Wellbore name,Well name,Drilling operator,Drilled in production licence,Status,Purpose,Purpose - planned,Content,Type,Subsea,Entered date,Completed date,Predrilled entry date,Predrilled completion date,Field,Drill permit,Discovery,Discovery wellbore,Kelly bushing elevation [m],Final vertical depth (TVD) [m RKB],Total depth (MD) [m RKB],Water depth [m],Kick off point [m RKB],Main area,Drilling facility,Drilling facility type,Drilling facility category,Production facility,Licensing activity awarded in,Multilateral,Content - planned,Entry year,Completed year,Reclassified from/to wellbore,Plugged and abondon date,Plugged date,Prod. licence for drilling target,Plot symbol,Geodetic datum,NS degrees,NS minutes,NS seconds,NS code,EW degrees,EW minutes,EW seconds,EW code,NS decimal degrees,EW decimal degrees,NS UTM [m],EW UTM [m],UTM zone,"Wellbore name, part 1","Wellbore name, part 2","Wellbore name, part 3","Wellbore name, part 4","Wellbore name, part 5","Wellbore name, part 6",FactPage url,Factmaps,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID production licence target,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD
0,1/3-A-1 H,1/3-A-1,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-07-22,2011-09-21,NaT,NaT,OSELVAR,3365-P,1/3-6 Oselvar,NO,45.0,3163.0,5927.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2011,None,NaT,NaT,NaN,50,ED50,56,55,55.06,N,2,40,16.66,E,56.931961,2.671294,6310001.50,479994.47,31,1,3,A,1,NaN,NaN,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6612,https://factmaps.npd.no/factmaps/3_0/?run=WellboreDevByNPDID&scale=100000&NPDID=6612,initial,,6612,43832.0,5506919.0,NaT,2013-09-21,2060266,NaN,278245.0,410592.0,0,2019-12-09,2015-10-06,16.02.2020
1,1/3-A-2 H,1/3-A-2,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-11-18,2012-01-19,2011-06-19,2011-07-04,OSELVAR,3366-P,1/3-6 Oselvar,NO,45.0,3170.0,5882.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2012,None,NaT,NaT,NaN,50,ED50,56,55,54.89,N,2,40,16.67,E,56.931914,2.671297,6309996.24,479994.61,31,1,3,A,2,NaN,NaN,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6613,https://factmaps.npd.no/factmaps/3_0/?run=WellboreDevByNPDID&scale=100000&NPDID=6613,initial,,6613,43832.0,5506919.0,NaT,2014-01-19,2060266,NaN,278245.0,410592.0,0,2019-12-09,2015-10-06,16.02.2020
2,1/3-A-3 H,1/3-A-3,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2012-03-04,2012-05-14,2011-07-05,2011-07-21,OSELVAR,3367-P,1/3-6 Oselvar,NO,45.0,3171.0,6665.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2012,2012,None,NaT,NaT,NaN,50,ED50,56,55,55.07,N,2,40,17.32,E,56.931964,2.671478,6310001.76,480005.63,31,1,3,A,3,NaN,NaN,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6614,https://factmaps.npd.no/factmaps/3_0/?run=WellboreDevByNPDID&scale=100000&NPDID=6614,initial,,6614,43832.0,5506919.0,NaT,2014-05-14,2060266,NaN,278245.0,410592.0,0,2019-12-09,2015-10-06,16.02.2020


<h4>QC - Which column headers are unique to Exploration or Development wells?</h4>

In [8]:
explo_columns = df_explo.columns.tolist()
dev_columns = df_dev.columns.tolist()

# List well headers unqiue to each dataframe
print('Attributes unique to Exploration wells:\n', sorted(set(explo_columns) - set(dev_columns)))
print('\nAttributes unique to Development wells:\n', sorted(set(dev_columns) - set(explo_columns)))

Attributes unique to Exploration wells:
 ['1st level with HC, age', '1st level with HC, formation', '2nd level with HC, age', '2nd level with HC, formation', '3rd level with HC, age', '3rd level with HC, formation', 'Bottom hole temperature [°C]', 'Drilling days', 'Maximum inclination [°]', 'NPDID site survey', 'Oldest penetrated age', 'Oldest penetrated formation', 'Pressrelease url', 'Reclassified date', 'Reentry', 'Reentry activity', 'Seismic location', 'Sitesurvey']

Attributes unique to Development wells:
 ['Content - planned', 'NPDID production facility', 'NPDID production licence target', 'Predrilled completion date', 'Predrilled entry date', 'Production facility']


<h4>Rename attributes for IC</h4>

In [9]:
# These are IC's default well header attributes (when matching columns in Import Well Header File)
# Try to use as many of these as possible when renaming below.
# Any other columns will need to be added to IC as Well Attributes.

# QUESTION - WHY IS THIS A DICT AND NOT LIST?

ic_default_attributes = {'Name', 'Code', 'Alternate 1', 'Alternate 2', 'API number', 'UWI number', 'Comment', 'Geodatum', 
                         'Longitude', 'Latitude', 'Grid system', 'Surface X', 'Surface Y', 'Elevation Reference',
                         'Elevation', 'KBE', 'RTE', 'DFE', 'GLE', 'SPUD date', 'Completion date', 'Status', 
                         'Quadrant', 'Block', 'Sub block', 'Field', 'Location', 'Operator', 'Country',
                         'Basin', 'Province', 'County', 'State', 'Section', 'Township', 'Range', 'Terminal depth',
                         'Water depth', 'Facility', 'Discovery name', 'Seismic line', 'Intent', 'Licence number'}

# Rename columns from/to. 
# Check spelling and capitalisation carefully when renaming to match IC's default attributes.

attributes_to_rename = {'Wellbore name' : 'Name',
                        'Well name' : 'Alternate 1',
                        'Drilling operator' : 'Operator',
                        'Drilled in production licence' : 'Licence number',
                        'Purpose' : 'Intent',
                        'Purpose - planned' : 'Intent - planned',
                        'Status' : 'Well status',
                        'Content' : 'Well content',
                        'Entered date' : 'SPUD date',
                        'Completed date' : 'Completion date',
                        'Discovery' : 'Discovery name',
                        'Seismic location' : 'Seismic line',
                        'Kelly bushing elevation [m]' : 'KBE',
                        'Total depth (MD) [m RKB]' : 'Terminal depth',
                        'Water depth [m]' : 'Water depth',
                        'Kick off  point [m RKB]' : 'Kick off point [m RKB]', #remove extra space
                        'Main area' : 'Location',
                        'Drilling facility' : 'Facility',
                        '1st level with HC, formation' : '1st level with HC formation', #remove commas to be csv friendly
                        '1st level with HC, age' : '1st level with HC age',
                        '2nd level with HC, formation' : '2nd level with HC formation',
                        '2nd level with HC, age' : '2nd level with HC age',
                        '3rd level with HC, formation' : '3rd level with HC formation',
                        '3rd level with HC, age' : '3rd level with HC age',
                        'Geodetic datum' : 'Geodatum',
                        'NS decimal degrees' : 'Latitude',
                        'EW decimal degrees' : 'Longitude',
                        'NS UTM [m]' : 'Surface Y',
                        'EW UTM [m]' : 'Surface X',
                        'Wellbore name, part 1' : 'Quadrant',
                        'Wellbore name, part 2' : 'Block', 
                        'Pressrelease url' : 'Press Release URL',
                        'FactPage url' : 'FactPage URL',
                        'Factmaps' : 'FactMaps URL'}

# Apply renaming to each of the dataframes
df_explo.rename(columns=attributes_to_rename, inplace=True)
df_dev.rename(columns=attributes_to_rename, inplace=True)

# QC only renamed columns
print("Renamed attributes only:")
renamed_columns = list(attributes_to_rename.values())
df_explo[renamed_columns].head(3)
#df_dev[renamed_columns].head(3)

Renamed attributes only:


,Name,Alternate 1,Operator,Licence number,Intent,Intent - planned,Well status,Well content,SPUD date,Completion date,Discovery name,Seismic line,KBE,Terminal depth,Water depth,Kick off point [m RKB],Location,Facility,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Geodatum,Latitude,Longitude,Surface Y,Surface X,Quadrant,Block,Press Release URL,FactPage URL,FactMaps URL
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,WILDCAT,P&A,OIL,1989-03-20,1989-06-04,1/2-1 Blane,PW 8303A - 10 SP. 290,22.0,3574.0,72.0,NaN,NORTH SEA,ROSS ISLE,FORTIES FM,PALEOCENE,NaN,NaN,,,ED50,56.887519,2.476583,6305128.26,468106.29,1,2,,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,WILDCAT,P&A,OIL SHOWS,2005-12-14,2006-02-02,None,inline 7429-trace 4824 Survey PGS CGMNOR,40.0,3434.0,74.0,NaN,NORTH SEA,MÆRSK GIANT,NaN,NaN,NaN,NaN,,,ED50,56.992222,2.496572,6316774.33,469410.10,1,2,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2006/1-2-2/,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=5192,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=5192
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,WILDCAT,P&A,GAS,1968-07-06,1968-11-11,1/3-1,LINE 5651 SP. E165,26.0,4877.0,71.0,NaN,NORTH SEA,ORION,TOR FM,LATE CRETACEOUS,CROMER KNOLL GP,EARLY CRETACEOUS,,,ED50,56.855833,2.851389,6301488.86,490936.87,1,3,,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=154,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=154


<h4>Delete some attributes we don't need in IC</h4>

In [10]:
# Coordinates are repeated elsewhere so we can delete the component parts from the dataframes.
# And we've renamed Wellbore name parts 1 and 2 to Quadrant and Block, and do not need the other parts.

attributes_to_drop = ['Plot symbol', 'NS degrees', 'NS minutes', 'NS seconds', 'NS code', 'EW degrees', 'EW minutes', 'EW seconds', 'EW code', 
                      'Wellbore name, part 3', 'Wellbore name, part 4', 'Wellbore name, part 5', 'Wellbore name, part 6']

df_explo.drop(attributes_to_drop, axis=1, inplace=True)
df_dev.drop(attributes_to_drop, axis=1, inplace=True)

print('Prove we still have well names and coordinates:')
df_explo[['Name', 'Latitude', 'Longitude', 'Surface Y', 'Surface X']].head(3)

Prove we still have well names and coordinates:


,Name,Latitude,Longitude,Surface Y,Surface X
0,1/2-1,56.887519,2.476583,6305128.26,468106.29
1,1/2-2,56.992222,2.496572,6316774.33,469410.10
2,1/3-1,56.855833,2.851389,6301488.86,490936.87


<h4>Truncate well list based on column and value(s)</h4>

In [11]:
# Enter the column and values you want to return, e.g. Location: BARENTS SEA, or Quadrant: 6204, 6205.
fltr_column = 'Location'

# List the names you want to *KEEP*!
fltr_value = ['NORTH SEA', 'NORWEGIAN SEA', 'BARENTS SEA']

# Apply the filter to the dataframes
indexNames = df_explo[~df_explo[fltr_column].isin(fltr_value)].index
df_explo.drop(indexNames , inplace=True)

indexNames = df_dev[~df_dev[fltr_column].isin(fltr_value)].index
df_dev.drop(indexNames , inplace=True)

# Get dataframe shape and unpack tuples
(exploRows, exploCols) = df_explo.shape
(devRows, devCols) = df_dev.shape

# Print out the results
print("After filtering on {}: {}, you are left with:\n {} rows for Exploration wells, and {} rows for Development wells."
      .format(fltr_column, fltr_value, exploRows, devRows))
print('The first and last rows are:')

# Print the first and last rows of the Exploration dataframe to check that the filter has worked
df_explo.iloc[[0, -1]]

After filtering on Location: ['NORTH SEA', 'NORWEGIAN SEA', 'BARENTS SEA'], you are left with:
 1933 rows for Exploration wells, and 5154 rows for Development wells.
The first and last rows are:


,Name,Alternate 1,Operator,Licence number,Intent,Well status,Well content,Type,Subsea,SPUD date,Completion date,Field,Drill permit,Discovery name,Discovery wellbore,Bottom hole temperature [°C],Sitesurvey,Seismic line,Maximum inclination [°],KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Location,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Intent - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Drilling days,Reentry,Prod. licence for drilling target,Plugged and abondon date,Plugged date,Geodatum,Latitude,Longitude,Surface Y,Surface X,UTM zone,Quadrant,Block,Press Release URL,FactPage URL,FactMaps URL,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,Reclassified date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,NPDID site survey,Date main level updated,Date all updated,Date sync NPD
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,1989-03-20,1989-06-04,BLANE,604-L,1/2-1 Blane,YES,147.0,None,PW 8303A - 10 SP. 290,2.0,22.0,NaN,3574.0,72.0,NaN,CAMPANIAN,TOR FM,NORTH SEA,ROSS ISLE,SEMISUB STEEL,MOVEABLE,12,NO,WILDCAT,1989,1989,None,None,FORTIES FM,PALEOCENE,NaN,NaN,,,77,NO,NaN,NaT,NaT,ED50,56.887519,2.476583,6305128.26,468106.29,31,1,2,,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382,initial,,2007-12-19,1991-06-04,NaT,1382,43814.0,3437650.0,296245.0,0,21956.0,NaN,2019-10-03,2019-10-03,16.02.2020
1932,7435/12-1,7435/12-1,Statoil Petroleum AS,859,WILDCAT,P&A,GAS,EXPLORATION,NO,2017-08-09,2017-09-01,None,1667-L,7435/12-1 (Korpfjell),YES,54.0,None,ST14005T15: Inline 7833. X-Line 7829,3.1,32.0,1539.0,1540.0,253.0,NaN,MIDDLE TRIASSIC,KOBBE FM,BARENTS SEA,SONGA ENABLER,SEMISUB STEEL,MOVEABLE,23,NO,WILDCAT,2017,2017,None,None,STØ FM,MIDDLE JURASSIC,KOBBE FM,MIDDLE TRIASSIC,,,24,NO,NaN,NaT,NaT,ED50,74.071725,35.808628,8222886.96,402277.88,37,7435,12,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2017/743512-1/,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=8228,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=8228,initial,,2019-09-01,2019-09-01,NaT,8228,29491696.0,NaN,439972.0,0,28169055.0,NaN,2019-12-07,2019-12-07,16.02.2020


<h4>CREATE FILES - create Reference files for IC containing URLs for Exploration and Development wells</h4>

In [12]:
# Converts three URL columns into three rows. Adds a Title column and sorts by Well and Title.
df_explo_references = df_explo[['Name', 'Press Release URL', 'FactPage URL', 'FactMaps URL']]
df_explo_references = pd.melt(df_explo_references, id_vars='Name', value_vars=['Press Release URL', 'FactPage URL', 'FactMaps URL'], var_name='Title', value_name='URL')
df_explo_references.sort_values(['Name', 'Title'], inplace=True)

# Remove empty rows, specifically where no 'Press Release URL' for Exploration references
df_explo_references['URL'].replace(' ', np.nan, inplace=True)
df_explo_references.dropna(subset=['URL'], inplace=True)

# Name and create file for Exploration wells
explo_ref_filename = 'output data/IC_explo_references.csv'
df_explo_references.to_csv(explo_ref_filename, index=False)
print('Created file:', explo_ref_filename)
df_explo_references.head(n=6)

Created file: output data/IC_explo_references.csv


,Name,Title,URL
3866,1/2-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
1933,1/2-1,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382
3867,1/2-2,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=5192
1934,1/2-2,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=5192
1,1/2-2,Press Release URL,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2006/1-2-2/
3868,1/3-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=154


In [13]:
# As above, but creates 'Reference' file for Development Wells (minus the Press Release URL)
df_dev_references = df_dev[['Name', 'FactPage URL', 'FactMaps URL']]
df_dev_references = pd.melt(df_dev_references, id_vars='Name', 
                            value_vars=['FactPage URL', 'FactMaps URL'], 
                            var_name='Title', value_name='URL')
df_dev_references.sort_values(['Name', 'Title'], inplace=True)

# Name and create file for Development wells
dev_ref_filename = 'output data/IC_dev_references.csv'
df_dev_references.to_csv(dev_ref_filename, index=False)
print('Created file:', dev_ref_filename)
df_dev_references.head(n=4)

Created file: output data/IC_dev_references.csv


,Name,Title,URL
5154,1/3-A-1 H,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreDevByNPDID&scale=100000&NPDID=6612
0,1/3-A-1 H,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6612
5155,1/3-A-2 H,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreDevByNPDID&scale=100000&NPDID=6613
1,1/3-A-2 H,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Development|All&nav3=6613


In [14]:
# Drop URL attributes
# Now that we've output the URLs to separate files, we no longer need them in the Exploration and Development dataframes.
df_explo.drop(['Press Release URL', 'FactPage URL', 'FactMaps URL'], axis=1, inplace=True)
df_dev.drop(['FactPage URL', 'FactMaps URL'], axis=1, inplace=True)

df_explo.head(3)

,Name,Alternate 1,Operator,Licence number,Intent,Well status,Well content,Type,Subsea,SPUD date,Completion date,Field,Drill permit,Discovery name,Discovery wellbore,Bottom hole temperature [°C],Sitesurvey,Seismic line,Maximum inclination [°],KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Location,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Intent - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Drilling days,Reentry,Prod. licence for drilling target,Plugged and abondon date,Plugged date,Geodatum,Latitude,Longitude,Surface Y,Surface X,UTM zone,Quadrant,Block,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,Reclassified date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,NPDID site survey,Date main level updated,Date all updated,Date sync NPD
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,1989-03-20,1989-06-04,BLANE,604-L,1/2-1 Blane,YES,147.0,None,PW 8303A - 10 SP. 290,2.0,22.0,NaN,3574.0,72.0,NaN,CAMPANIAN,TOR FM,NORTH SEA,ROSS ISLE,SEMISUB STEEL,MOVEABLE,12,NO,WILDCAT,1989,1989,None,None,FORTIES FM,PALEOCENE,NaN,NaN,,,77,NO,NaN,NaT,NaT,ED50,56.887519,2.476583,6305128.26,468106.29,31,1,2,initial,,2007-12-19,1991-06-04,NaT,1382,43814.0,3437650.0,296245.0,0,21956.0,NaN,2019-10-03,2019-10-03,16.02.2020
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P&A,OIL SHOWS,EXPLORATION,NO,2005-12-14,2006-02-02,None,1103-L,None,NO,138.0,None,inline 7429-trace 4824 Survey PGS CGMNOR,4.9,40.0,3432.0,3434.0,74.0,NaN,PALEOCENE,EKOFISK FM,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,12,NO,WILDCAT,2005,2006,None,None,NaN,NaN,NaN,NaN,,,51,NO,NaN,NaT,NaT,ED50,56.992222,2.496572,6316774.33,469410.10,31,1,2,initial,,2008-08-15,2008-02-02,NaT,5192,NaN,NaN,278245.0,0,2424919.0,NaN,2019-10-03,2019-10-03,16.02.2020
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P&A,GAS,EXPLORATION,NO,1968-07-06,1968-11-11,None,15-L,1/3-1,YES,182.0,None,LINE 5651 SP. E165,18.0,26.0,NaN,4877.0,71.0,NaN,LATE PERMIAN,ZECHSTEIN GP,NORTH SEA,ORION,JACK-UP 3 LEGS,MOVEABLE,1-A,NO,WILDCAT,1968,1968,None,None,TOR FM,LATE CRETACEOUS,CROMER KNOLL GP,EARLY CRETACEOUS,,,129,NO,NaN,NaT,NaT,ED50,56.855833,2.851389,6301488.86,490936.87,31,1,3,initial,,2010-04-30,1970-11-11,NaT,154,43820.0,NaN,288604.0,0,20844.0,NaN,2019-10-03,2019-10-03,16.02.2020


In [15]:
# Add new column(s) and assign constant value, e.g. Country: NORWAY.
df_explo['Country'] = 'NORWAY' 
df_dev['Country'] = 'NORWAY'

# IC Version 4.3.1 and earlier only. Fixed in 4.3.2.
# First lets rename an extraordinarily long string in column 'Seismic line' to avoid an error in IC.
#df_explo['Seismic line'] = df_explo['Seismic line'].replace('TUN15M01 3D bin datasett: Inline reference: 12688 Croslline reference: between 12383 and 12384', 'TUN15M01 3D bin: Inline 12688 Crossline 12383-12384')

# Remove decimal places introduced to the 'NPDIP' columns
df_explo['NPDID discovery'] = df_explo['NPDID discovery'].fillna(0).astype(int)
df_dev['NPDID discovery'] = df_dev['NPDID discovery'].fillna(0).astype(int)

df_explo['NPDID drilling facility'] = df_explo['NPDID drilling facility'].fillna(0).astype(int)
df_dev['NPDID drilling facility'] = df_dev['NPDID drilling facility'].fillna(0).astype(int)

df_explo['NPDID field'] = df_explo['NPDID field'].fillna(0).astype(int)
df_dev['NPDID field'] = df_dev['NPDID field'].fillna(0).astype(int)

# Copy data from one column to another, preserving the original.
df_explo['UWI number'] = df_explo['NPDID wellbore']
df_dev['UWI number'] = df_dev['NPDID wellbore']

# Check the result
df_explo[['Name', 'Country', 'NPDID drilling facility', 'NPDID wellbore']].head(3)

,Name,Country,NPDID drilling facility,NPDID wellbore
0,1/2-1,NORWAY,296245,1382
1,1/2-2,NORWAY,278245,5192
2,1/3-1,NORWAY,288604,154


<h4>Concatenate Well Status & Well Content to match IC's Well Symbols dictionary</h4>

In [16]:
# This cell creates a new column called 'Status', combining 'Well Status' and 'Well Content'
# These values should match IC's Well Symbols graphic dictionary entries, e.g. "P & A Oil Shows"

# Change 'P&A' to 'P & A'.
df_explo['Well status'] = df_explo['Well status'].replace(to_replace='P&A', value='P & A')
# First letter of each word capitalised
df_explo['Status'] = df_explo['Well status'].str.title() + ' ' + df_explo['Well content'].str.title()

# As above but for Development wells
df_dev['Well status'] = df_dev['Well status'].replace(to_replace='P&A', value='P & A')
df_dev['Status'] = df_dev['Well status'].str.title() + ' ' + df_dev['Well content'].str.title()

# Replace a few other things to help with matching
df_explo = df_explo.replace({'Status' : { ' Not Available' : '', ' Not Applicable' : '', '/' : ' ', 'Oil Gas ' : 'Oil & Gas '}}, regex=True)
df_dev = df_dev.replace({'Status' :     { ' Not Available' : '', ' Not Applicable' : '', '/' : ' ', 'Oil Gas ' : 'Oil & Gas '}}, regex=True)

# Rename Status to status? (links to symbol_id??? e.g. 22)

# Check the results
df_explo[['Name', 'Status']].head(n=10)
#df_dev[['Name', 'Status']].tail(n=10)

,Name,Status
0,1/2-1,P & A Oil
1,1/2-2,P & A Oil Shows
2,1/3-1,P & A Gas
3,1/3-2,P & A Dry
4,1/3-3,P & A Oil
5,1/3-4,P & A Oil Shows
6,1/3-5,P & A Dry
7,1/3-6,P & A Gas Condensate
8,1/3-7,P & A Oil
9,1/3-8,P & A Shows


In [17]:
# List all unique entries under Status for all wells.
# In IC, open Database > Graphic Dictionaries > Well Symbols, and ensure you have dictionary entries for each.

lst_explo_status = sorted(set(df_explo['Status'].astype(str)))
lst_dev_status = sorted(set(df_dev['Status'].astype(str)))

lst_all_status = lst_explo_status + lst_dev_status

print("{} unique status values to include in IC 'Well Symbols' graphic dictionary:".format(len(lst_all_status)))
print('')
lst_unique_status = sorted(set(lst_all_status))
print(', '.join(lst_unique_status))

93 unique status values to include in IC 'Well Symbols' graphic dictionary:

Blowout Gas Shows, Closed, Closed Cuttings, Closed Gas, Closed Gas Condensate, Closed Oil, Closed Oil Gas, Closed Oil Gas Condensate, Closed Water, Closed Water Gas, Drilling Dry, Injecting Co2, Injecting Cuttings, Injecting Gas, Injecting Oil, Injecting Water, Injecting Water Gas, Junked, Junked Dry, Junked Oil, Junked Oil Gas, Junked Oil Gas Shows, Junked Shows, Junked Water, P & A, P & A Cuttings, P & A Dry, P & A Gas, P & A Gas Condensate, P & A Gas Shows, P & A Oil, P & A Oil Gas, P & A Oil Gas Condensate, P & A Oil Gas Shows, P & A Oil Shows, P & A Shows, P & A Water, Plugged, Plugged Cuttings, Plugged Dry, Plugged Gas, Plugged Gas Condensate, Plugged Oil, Plugged Oil Gas, Plugged Oil Gas Condensate, Plugged Water, Plugged Water Gas, Predrilled, Predrilled Gas, Producing Gas, Producing Gas Condensate, Producing Oil, Producing Oil Gas, Producing Oil Gas Condensate, Producing Water, Producing Water Gas, Re

<h4>Concatenate cells to create 'Grid system' in IC format</h4>

In [18]:
# At time of writing, there are several problems with 'Geodatum' in the NPD datasets, including:
#  - trailing spaces ('ED50 ') in all Explo wells
#  - erroneous '56ED50', '60ED50' and '61ED50' values in Dev wells
#  - missing 'ED50' values in two explo wells
# Luckily, we can just force 'ED50' on all these cells!

df_explo['Geodatum'] = 'ED50'
df_dev['Geodatum'] = 'ED50'

# Concatenate cells to create a new column 'Grid system' in IC format (e.g. "ED50 / UTM Zone 31N")

df_explo['Grid system'] = df_explo['Geodatum'] + ' / ' + 'UTM zone ' + df_explo['UTM zone'].map(str) + 'N'
df_dev['Grid system'] = df_dev['Geodatum'] + ' / ' + 'UTM zone ' + df_dev['UTM zone'].map(str) + 'N'

print('Geodatum and Grid systems for IC:')
df_explo[['Name', 'Geodatum', 'Grid system']].head(3)

Geodatum and Grid systems for IC:


,Name,Geodatum,Grid system
0,1/2-1,ED50,ED50 / UTM zone 31N
1,1/2-2,ED50,ED50 / UTM zone 31N
2,1/3-1,ED50,ED50 / UTM zone 31N


<h4>QC - check well headers</h4>

In [19]:
# Print out attributes lists, reflecting all the changes above.
# Use these lists to check the current order of your columns in each, and consider how you might like to re-order them.
# Any columns created above (including: Country, Status, Grid system) currently appear at the end of the lists.

print('--- BEFORE RE-ORDERING ---\n')
print(len(df_explo.columns), 'Exploration attributes:\n', list(df_explo.columns), '\n')
print(len(df_dev.columns), 'Development attributes:\n', list(df_dev.columns))

--- BEFORE RE-ORDERING ---

75 Exploration attributes:
 ['Name', 'Alternate 1', 'Operator', 'Licence number', 'Intent', 'Well status', 'Well content', 'Type', 'Subsea', 'SPUD date', 'Completion date', 'Field', 'Drill permit', 'Discovery name', 'Discovery wellbore', 'Bottom hole temperature [°C]', 'Sitesurvey', 'Seismic line', 'Maximum inclination [°]', 'KBE', 'Final vertical depth (TVD) [m RKB]', 'Terminal depth', 'Water depth', 'Kick off point [m RKB]', 'Oldest penetrated age', 'Oldest penetrated formation', 'Location', 'Facility', 'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 'Multilateral', 'Intent - planned', 'Entry year', 'Completed year', 'Reclassified from/to wellbore', 'Reentry activity', '1st level with HC formation', '1st level with HC age', '2nd level with HC formation', '2nd level with HC age', '3rd level with HC formation', '3rd level with HC age', 'Drilling days', 'Reentry', 'Prod. licence for drilling target', 'Plugged and abond

<h4>Re-order all columns (OPTIONAL)</h4>

In [20]:
# # Specifies the order of columns for Exploration and Development wells in the final outputs.
# # It's not compulsory to re-order columns, as IC lists all non-default attributes alphabetically.

# explo_order = ['Name', 'Alternate 1', 'UWI number', 'Quadrant', 'Block', 'Operator', 'Licence number', 'Intent', 
#                 'Intent - planned', 'Well status', 'Well content', 'Status', 'Type', 'Subsea', 'SPUD date', 
#                 'Completion date', 'Field', 'Drill permit', 'Discovery name', 'Discovery wellbore', 
#                 'Bottom hole temperature [°C]', 'Seismic line', 'Maximum inclination [°]', 'KBE', 
#                 'Final vertical depth (TVD) [m RKB]', 'Terminal depth', 'Water depth', 'Kick off point [m RKB]', 
#                 'Oldest penetrated age', 'Oldest penetrated formation', 'Location', 'Country', 'Facility', 
#                 'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 
#                 'Multilateral', 'Entry year', 'Completed year', 'Reclassified from/to wellbore', 'Reentry activity', 
#                 'Plot symbol', '1st level with HC formation', '1st level with HC age', '2nd level with HC formation', 
#                 '2nd level with HC age', '3rd level with HC formation', '3rd level with HC age', 'Drilling days', 
#                 'Reentry', 'Geodatum', 'Latitude', 'Longitude', 'Surface X', 'Surface Y', 'UTM zone', 'Grid system', 
#                 'DISKOS Well Type', 'DISKOS Wellbore Parent', 
#                 'Publication date', 'Release date', 'NPDID wellbore', 'NPDID discovery', 'NPDID field', 
#                 'NPDID drilling facility', 'NPDID wellbore reclassified from', 'NPDID production licence drilled in', 
#                 'Date main level updated', 'Date all updated', 'Date sync NPD']

# dev_order = ['Name', 'Alternate 1', 'UWI number', 'Quadrant', 'Block', 'Operator', 'Licence number', 'Intent', 
#               'Intent - planned', 'Well status', 'Well content',  'Status', 'Content - planned', 'Type', 'Subsea',
#               'SPUD date', 'Completion date', 'Field', 'Predrilled entry date','Predrilled completion date', 
#               'Drill permit', 'Discovery name', 'Discovery wellbore', 'KBE', 'Final vertical depth (TVD) [m RKB]',
#               'Terminal depth', 'Water depth', 'Kick off point [m RKB]', 'Location', 'Country', 'Facility', 
#               'Drilling facility type', 'Drilling facility category', 'Licensing activity awarded in', 
#               'Production facility', 'Multilateral', 'Entry year', 'Completed year','Reclassified from/to wellbore', 
#               'Plot symbol', 'Geodatum', 'Latitude', 'Longitude', 'Surface Y', 'Surface X', 'UTM zone',  'Grid system', 
#               'DISKOS Well Type', 'DISKOS Wellbore Parent', 'NPDID wellbore', 
#               'NPDID discovery', 'NPDID field', 'Publication date', 'Release date', 'NPDID production licence drilled in', 
#               'NPDID drilling facility', 'NPDID production facility','NPDID wellbore reclassified from', 
#               'Date main level updated', 'Date all updated', 'Date sync NPD']

In [21]:
# # Check if your list of re-ordered attributes is complete.
# missing_explo = set(df_explo.columns).difference(explo_order)
# missing_dev = set(df_dev.columns).difference(dev_order)

# if len(missing_explo) > 0:
#     print('Your re-ordered list of Exploration attributes is incomplete. You must include:\n {}.\n'.format(missing_explo))
# else:
#     print('Your re-ordered list of Exploration attributes is complete.\n')
    
# if len(missing_dev) > 0:
#     print('Your re-ordered list of Development attributes is incomplete. You must include:\n {}.'.format(missing_dev))
# else:
#     print('Your re-ordered list of Development attributes is complete.')

In [22]:
# # Only when your re-ordered lists of Exploration and Development attributes are complete should you run this cell,
# # Otherwise these will not be included in the output file!
# # Applies the re-ordering to the dataframes

# df_explo = df_explo.reindex(columns=explo_order)
# df_dev = df_dev.reindex(columns=dev_order)

<h4>QC column values</h4>

In [23]:
# Print out all unique values for selected attributes (example: Operator and Field)

def lstheaderfields (*args):
    for arg in args:
        print('---' , arg, '---')
        print('')
        words = [x for x in df_explo[arg].unique()]
        print('Exploration wells:')
        print(words)
        print('')
        words = [x for x in df_dev[arg].unique()]
        print('Development wells:')
        print(words)
        print("")
        
# Enter the names of columns you would like to check
lstheaderfields('Operator', 'Field')

--- Operator ---

Exploration wells:
['Phillips Petroleum Norsk AS', 'Paladin Resources Norge AS', 'A/S Norske Shell', 'Elf Petroleum Norge AS', 'Amoco Norway Oil Company', 'BP Norway Limited U.A.', 'DONG E&P Norge AS', 'BG Norge AS', 'Phillips Petroleum Company Norway', 'Conoco Norway Inc.', 'Amerada Hess Norge AS', 'Total E&P Norge AS', 'Den norske stats oljeselskap a.s', 'BP Petroleum Dev. of Norway AS', 'Talisman Energy Norge AS', 'Det norske oljeselskap ASA', 'Aker BP ASA', 'Saga Petroleum ASA', 'Norske Murphy Oil Company', 'Norwegian Gulf Exploration Company AS', 'Norsk Hydro Produksjon AS', 'ConocoPhillips Skandinavia AS', 'Statoil Petroleum AS', 'Norsk Agip AS', 'StatoilHydro Petroleum AS', 'Lundin Norway AS', 'MOL Norge AS', 'Faroe Petroleum Norge AS', 'Edison Norge AS', 'Elf Norge A/S', 'Premier Oil Norge AS', 'Repsol Exploration Norge AS', 'LOTOS Exploration and Production Norge AS', 'Esso Exploration and Production Norway A/S', 'Unocal Norge A/S', 'Centrica Resources (Norge

<h4>CREATE FILES - create well header files for exploration and development wells</h4>

In [24]:
# Output exploration well headers

print('{} exploration wells from {} to {}.'.format(len(df_explo), 
                                                   df_explo['Name'][df_explo.index[0]], 
                                                   df_explo['Name'][df_explo.index[-1]]))

output_to_csv(file='IC_wellbore_exploration_all', df=df_explo)

1933 exploration wells from 1/2-1 to 7435/12-1.
Saved to: output data/IC_wellbore_exploration_all.csv


,Name,Alternate 1,Operator,Licence number,Intent,Well status,Well content,Type,Subsea,SPUD date,Completion date,Field,Drill permit,Discovery name,Discovery wellbore,Bottom hole temperature [°C],Sitesurvey,Seismic line,Maximum inclination [°],KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Oldest penetrated age,Oldest penetrated formation,Location,Facility,Drilling facility type,Drilling facility category,Licensing activity awarded in,Multilateral,Intent - planned,Entry year,Completed year,Reclassified from/to wellbore,Reentry activity,1st level with HC formation,1st level with HC age,2nd level with HC formation,2nd level with HC age,3rd level with HC formation,3rd level with HC age,Drilling days,Reentry,Prod. licence for drilling target,Plugged and abondon date,Plugged date,Geodatum,Latitude,Longitude,Surface Y,Surface X,UTM zone,Quadrant,Block,DISKOS Well Type,DISKOS Wellbore Parent,Publication date,Release date,Reclassified date,NPDID wellbore,NPDID discovery,NPDID field,NPDID drilling facility,NPDID wellbore reclassified from,NPDID production licence drilled in,NPDID site survey,Date main level updated,Date all updated,Date sync NPD,Country,UWI number,Status,Grid system
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P & A,OIL,EXPLORATION,NO,1989-03-20,1989-06-04,BLANE,604-L,1/2-1 Blane,YES,147.0,NaN,PW 8303A - 10 SP. 290,2.0,22.0,NaN,3574.0,72.0,NaN,CAMPANIAN,TOR FM,NORTH SEA,ROSS ISLE,SEMISUB STEEL,MOVEABLE,12,NO,WILDCAT,1989,1989,NaN,NaN,FORTIES FM,PALEOCENE,NaN,NaN,,,77,NO,NaN,NaN,NaN,ED50,56.887519,2.476583,6305128.26,468106.29,31,1,2,initial,,2007-12-19,1991-06-04,NaN,1382,43814,3437650,296245,0,21956.0,NaN,2019-10-03,2019-10-03,16.02.2020,NORWAY,1382,P & A Oil,ED50 / UTM zone 31N
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P & A,OIL SHOWS,EXPLORATION,NO,2005-12-14,2006-02-02,NaN,1103-L,NaN,NO,138.0,NaN,inline 7429-trace 4824 Survey PGS CGMNOR,4.9,40.0,3432.0,3434.0,74.0,NaN,PALEOCENE,EKOFISK FM,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,12,NO,WILDCAT,2005,2006,NaN,NaN,NaN,NaN,NaN,NaN,,,51,NO,NaN,NaN,NaN,ED50,56.992222,2.496572,6316774.33,469410.10,31,1,2,initial,,2008-08-15,2008-02-02,NaN,5192,0,0,278245,0,2424919.0,NaN,2019-10-03,2019-10-03,16.02.2020,NORWAY,5192,P & A Oil Shows,ED50 / UTM zone 31N
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P & A,GAS,EXPLORATION,NO,1968-07-06,1968-11-11,NaN,15-L,1/3-1,YES,182.0,NaN,LINE 5651 SP. E165,18.0,26.0,NaN,4877.0,71.0,NaN,LATE PERMIAN,ZECHSTEIN GP,NORTH SEA,ORION,JACK-UP 3 LEGS,MOVEABLE,1-A,NO,WILDCAT,1968,1968,NaN,NaN,TOR FM,LATE CRETACEOUS,CROMER KNOLL GP,EARLY CRETACEOUS,,,129,NO,NaN,NaN,NaN,ED50,56.855833,2.851389,6301488.86,490936.87,31,1,3,initial,,2010-04-30,1970-11-11,NaN,154,43820,0,288604,0,20844.0,NaN,2019-10-03,2019-10-03,16.02.2020,NORWAY,154,P & A Gas,ED50 / UTM zone 31N


In [25]:
# Output development well headers

print('{} development wells {} to {}.'.format(len(df_dev), 
                                                   df_dev['Name'][df_dev.index[0]], 
                                                   df_dev['Name'][df_dev.index[-1]]))

output_to_csv(file='IC_wellbore_development_all', df=df_dev)

5154 development wells 1/3-A-1 H to 7122/10-I-4 H.
Saved to: output data/IC_wellbore_development_all.csv


,Name,Alternate 1,Operator,Licence number,Well status,Intent,Intent - planned,Well content,Type,Subsea,SPUD date,Completion date,Predrilled entry date,Predrilled completion date,Field,Drill permit,Discovery name,Discovery wellbore,KBE,Final vertical depth (TVD) [m RKB],Terminal depth,Water depth,Kick off point [m RKB],Location,Facility,Drilling facility type,Drilling facility category,Production facility,Licensing activity awarded in,Multilateral,Content - planned,Entry year,Completed year,Reclassified from/to wellbore,Plugged and abondon date,Plugged date,Prod. licence for drilling target,Geodatum,Latitude,Longitude,Surface Y,Surface X,UTM zone,Quadrant,Block,DISKOS Well Type,DISKOS Wellbore Parent,NPDID wellbore,NPDID discovery,NPDID field,Publication date,Release date,NPDID production licence drilled in,NPDID production licence target,NPDID drilling facility,NPDID production facility,NPDID wellbore reclassified from,Date main level updated,Date all updated,Date sync NPD,Country,UWI number,Status,Grid system
0,1/3-A-1 H,1/3-A-1,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-07-22 00:00:00.000,2011-09-21 00:00:00.000,NaN,NaN,OSELVAR,3365-P,1/3-6 Oselvar,NO,45.0,3163.0,5927.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2011,NaN,NaN,NaN,NaN,ED50,56.931961,2.671294,6310001.50,479994.47,31,1,3,initial,,6612,43832,5506919,NaN,2013-09-21 00:00:00.000,2060266,NaN,278245,410592.0,0,2019-12-09 00:00:00.000,2015-10-06 00:00:00.000,16.02.2020,NORWAY,6612,Closed Oil,ED50 / UTM zone 31N
1,1/3-A-2 H,1/3-A-2,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2011-11-18 00:00:00.000,2012-01-19 00:00:00.000,2011-06-19 00:00:00.000,2011-07-04 00:00:00.000,OSELVAR,3366-P,1/3-6 Oselvar,NO,45.0,3170.0,5882.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2011,2012,NaN,NaN,NaN,NaN,ED50,56.931914,2.671297,6309996.24,479994.61,31,1,3,initial,,6613,43832,5506919,NaN,2014-01-19 00:00:00.000,2060266,NaN,278245,410592.0,0,2019-12-09 00:00:00.000,2015-10-06 00:00:00.000,16.02.2020,NORWAY,6613,Closed Oil,ED50 / UTM zone 31N
2,1/3-A-3 H,1/3-A-3,DONG E&P Norge AS,274,CLOSED,PRODUCTION,PRODUCTION,OIL,DEVELOPMENT,YES,2012-03-04 00:00:00.000,2012-05-14 00:00:00.000,2011-07-05 00:00:00.000,2011-07-21 00:00:00.000,OSELVAR,3367-P,1/3-6 Oselvar,NO,45.0,3171.0,6665.0,72.0,NaN,NORTH SEA,MÆRSK GIANT,JACK-UP 3 LEGS,MOVEABLE,OSELVAR,NST2001,NO,OIL,2012,2012,NaN,NaN,NaN,NaN,ED50,56.931964,2.671478,6310001.76,480005.63,31,1,3,initial,,6614,43832,5506919,NaN,2014-05-14 00:00:00.000,2060266,NaN,278245,410592.0,0,2019-12-09 00:00:00.000,2015-10-06 00:00:00.000,16.02.2020,NORWAY,6614,Closed Oil,ED50 / UTM zone 31N


<h4>Well Attributes to create in IC</h4>

In [26]:
# The following attributes are not IC defaults and need to be created under Wells > Attributes.
# Alternatively, use the SQL code produced in the next cell to create these rows in SSMS. 

# Find the full list of attributes after all the editing you've done above
all_attributes = set(list(df_explo.columns) + list(df_dev.columns))

# Find and count those attributes you'll need to create in IC
non_default_attributes = list(set(all_attributes).difference(ic_default_attributes))
non_default_attributes.sort()
num_non_default_attributes = len(non_default_attributes)

print('The following {} attributes are not IC defaults and must be added to IC:\n'.format(num_non_default_attributes))
print(list(non_default_attributes))

The following 55 attributes are not IC defaults and must be added to IC:

['1st level with HC age', '1st level with HC formation', '2nd level with HC age', '2nd level with HC formation', '3rd level with HC age', '3rd level with HC formation', 'Bottom hole temperature [°C]', 'Completed year', 'Content - planned', 'DISKOS Well Type', 'DISKOS Wellbore Parent', 'Date all updated', 'Date main level updated', 'Date sync NPD', 'Discovery wellbore', 'Drill permit', 'Drilling days', 'Drilling facility category', 'Drilling facility type', 'Entry year', 'Final vertical depth (TVD) [m RKB]', 'Intent - planned', 'Kick off point [m RKB]', 'Licensing activity awarded in', 'Maximum inclination [°]', 'Multilateral', 'NPDID discovery', 'NPDID drilling facility', 'NPDID field', 'NPDID production facility', 'NPDID production licence drilled in', 'NPDID production licence target', 'NPDID site survey', 'NPDID wellbore', 'NPDID wellbore reclassified from', 'Oldest penetrated age', 'Oldest penetrated formatio

In [27]:
# If you have database administration privileges, you can use this cell 
# to generate the SQL Query code that will create Well Attributes in IC in the format:
    
    #INSERT INTO t_WellsUserFields (f_FieldId, f_FieldName, f_IsInputUsed, f_InputID, f_Description, f_Origin, f_SortOrder)
        #VALUES (1, 'Attribute', 'False', 0, 'Description of attribute', 0, 1);

        # This assumes you have no yet created any Well Attributes in IC. 
        # If you have already, you'll need to tweak the 3 variables below.

pk_index = 0 #Enter one less than your highest pk_index
original_pk_index = 0 #Enter the same number as above (this one we won't change)
f_sortorder = 0 #Enter the next appropriate f_sortorder

print("INSERT INTO t_WellsUserFields")
print("  (f_FieldId, f_FieldName, f_IsInputUsed, f_InputID, f_Description, f_Origin, f_SortOrder)")
print("VALUES")

for i in non_default_attributes:
    pk_index += 1
    f_sortorder += 1
    if pk_index < (num_non_default_attributes + original_pk_index):
        print("  ({x}, '{y}', 'False', 0, 'Userfield {y}', 0, {z}),".format(x = pk_index, y = i, z = f_sortorder))
    else:
        print("  ({x}, '{y}', 'False', 0, 'Userfield {y}', 0, {z});".format(x = pk_index, y = i, z = f_sortorder))

# Follow these steps:
    # 1. Open your IC database in SQL Server Management Studio. IC must be closed/computer restarted to open a LocalDB in SSMS.
    # 2. Expand 'Tables', scroll down to 't_WellsUserFields' and right-click 'Edit Top 200 Rows'.
    # 3. Press Ctrl+N to create a new query, copy and paste the following SQL code to the blank query and hit F5.

INSERT INTO t_WellsUserFields
  (f_FieldId, f_FieldName, f_IsInputUsed, f_InputID, f_Description, f_Origin, f_SortOrder)
VALUES
  (1, '1st level with HC age', 'False', 0, 'Userfield 1st level with HC age', 0, 1),
  (2, '1st level with HC formation', 'False', 0, 'Userfield 1st level with HC formation', 0, 2),
  (3, '2nd level with HC age', 'False', 0, 'Userfield 2nd level with HC age', 0, 3),
  (4, '2nd level with HC formation', 'False', 0, 'Userfield 2nd level with HC formation', 0, 4),
  (5, '3rd level with HC age', 'False', 0, 'Userfield 3rd level with HC age', 0, 5),
  (6, '3rd level with HC formation', 'False', 0, 'Userfield 3rd level with HC formation', 0, 6),
  (7, 'Bottom hole temperature [°C]', 'False', 0, 'Userfield Bottom hole temperature [°C]', 0, 7),
  (8, 'Completed year', 'False', 0, 'Userfield Completed year', 0, 8),
  (9, 'Content - planned', 'False', 0, 'Userfield Content - planned', 0, 9),
  (10, 'DISKOS Well Type', 'False', 0, 'Userfield DISKOS Well Type', 0, 10),
  

<h4>Ensure the correct co-ordinate systems are added to your IC project</h4>

In [28]:
# In IC, open Project > Properties > Coords > Coordinate Systems
# Ensure each of the following co-ordinate system are installed **before importing well headers**

lstfield = sorted(set(df_explo['Geodatum'].astype(str)))
print('Geodatum:', ', '.join(lstfield))

lstfield = sorted(set(df_explo['Grid system'].astype(str)))
print('Grid systems:', ', '.join(lstfield))

Geodatum: ED50
Grid systems: ED50 / UTM zone 31N, ED50 / UTM zone 32N, ED50 / UTM zone 33N, ED50 / UTM zone 34N, ED50 / UTM zone 35N, ED50 / UTM zone 36N, ED50 / UTM zone 37N


<h4>Import the data to IC</h4>

In [29]:
# Before importing data to IC, ensure you have followed the last few steps to:
# - Create the appropriate Well Attributes in your IC Database.
# - Add the correct coordinate systems to your IC Project.

# Import reference files via Import > Well References
# Import well headers via Import > Headers.

# Note that, while the well header data imports very quickly, IC is a bit slow to create the wells if they don't already exist. Patience!

<h3>Core (Core Interval)</h3>

In [30]:
if data_source == 'web':
    df_core = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_core&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=165.225.81.99&CultureCode=en')

if data_source == 'file':
    df_core = pd.read_excel('input data/wellbore_core.xlsx')
    
df_core.head(3)

,Wellbore,Core sample number,Core sample - top depth,Core sample - bottom depth,Core sample depth - uom,Total core sample length [m],Number of cores samples,Cores available for sampling?,NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,1,10208.0,10208.4,[ft ],56.2,8,YES,1382,2016-05-31,16.02.2020
1,1/2-1,2,10216.0,10233.0,[ft ],56.2,8,YES,1382,2016-05-31,16.02.2020
2,1/2-1,3,0.0,0.0,None,56.2,8,YES,1382,2016-05-31,16.02.2020


In [31]:
df_core = df_core.replace(0, np.nan)
df_core.head(3)

,Wellbore,Core sample number,Core sample - top depth,Core sample - bottom depth,Core sample depth - uom,Total core sample length [m],Number of cores samples,Cores available for sampling?,NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,1,10208.0,10208.4,[ft ],56.2,8.0,YES,1382,2016-05-31,16.02.2020
1,1/2-1,2,10216.0,10233.0,[ft ],56.2,8.0,YES,1382,2016-05-31,16.02.2020
2,1/2-1,3,NaN,NaN,None,56.2,8.0,YES,1382,2016-05-31,16.02.2020


In [32]:
df_core.isnull().sum()

Wellbore                           0
Core sample number                 0
Core sample - top depth          136
Core sample -  bottom depth      136
Core sample depth - uom          130
Total core sample length [m]       5
Number of cores samples            6
Cores available for sampling?      0
NPDID wellbore                     0
Date updated                       0
Date sync NPD                      0
dtype: int64

In [33]:
filt = (df_core['Core sample - top depth'].isnull()) | (df_core['Core sample -  bottom depth'].isnull()) | (df_core['Core sample depth - uom'].isnull())

# (df_core['Core sample - top depth'] == 0.0) | (df_core['Core sample -  bottom depth'] == 0.0) |

df_core[filt].count()
df_core[filt]

,Wellbore,Core sample number,Core sample - top depth,Core sample - bottom depth,Core sample depth - uom,Total core sample length [m],Number of cores samples,Cores available for sampling?,NPDID wellbore,Date updated,Date sync NPD
2,1/2-1,3,NaN,NaN,None,56.20,8.0,YES,1382,2016-05-31,16.02.2020
341,15/9-11,8,NaN,NaN,None,104.30,11.0,YES,329,2016-05-31,16.02.2020
531,16/1-1,9,NaN,NaN,None,171.00,18.0,NO,147,2018-03-16,16.02.2020
889,2/11-9,2,NaN,NaN,None,33.25,4.0,YES,2153,2016-05-31,16.02.2020
903,2/11-A-2,6,NaN,NaN,None,126.14,10.0,YES,1616,2016-05-31,16.02.2020
...,...,...,...,...,...,...,...,...,...,...,...
7243,6507/8-2,2,NaN,NaN,None,24.60,3.0,YES,1076,2016-05-31,16.02.2020
7324,6608/10-4,1,NaN,NaN,None,145.57,8.0,YES,2256,2016-05-31,16.02.2020
7944,7128/6-1,4,NaN,NaN,None,443.70,24.0,YES,1836,2016-05-31,16.02.2020
7945,7128/6-1,5,NaN,NaN,None,443.70,24.0,YES,1836,2016-05-31,16.02.2020


In [34]:
df_core = df_core[~filt]
df_core

,Wellbore,Core sample number,Core sample - top depth,Core sample - bottom depth,Core sample depth - uom,Total core sample length [m],Number of cores samples,Cores available for sampling?,NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,1,10208.0,10208.40,[ft ],56.20,8.0,YES,1382,2016-05-31,16.02.2020
1,1/2-1,2,10216.0,10233.00,[ft ],56.20,8.0,YES,1382,2016-05-31,16.02.2020
3,1/2-1,4,10256.0,10286.00,[ft ],56.20,8.0,YES,1382,2016-05-31,16.02.2020
4,1/2-1,5,10286.0,10358.00,[ft ],56.20,8.0,YES,1382,2016-05-31,16.02.2020
5,1/2-1,6,10358.0,10364.60,[ft ],56.20,8.0,YES,1382,2016-05-31,16.02.2020
...,...,...,...,...,...,...,...,...,...,...,...
8273,9/2-A-4,2,4648.5,4676.70,[m ],68.00,3.0,YES,2828,2016-05-31,16.02.2020
8274,9/2-A-4,3,4676.7,4704.36,[m ],68.00,3.0,YES,2828,2016-05-31,16.02.2020
8275,9/3-1,1,1798.0,1805.40,[m ],7.40,1.0,YES,921,2016-05-31,16.02.2020
8276,9/8-1,1,1926.0,1933.35,[m ],23.77,2.0,YES,145,2016-05-31,16.02.2020


In [35]:
#df_core.isnull().sum()

In [36]:
df_core['Core sample depth - uom'].unique()

array(['[ft  ]', '[m   ]'], dtype=object)

In [37]:
for index, row in df_core.iterrows():
    if row['Core sample depth - uom'] == '[ft  ]':
        df_core.loc[index, 'Top depth'] = (row['Core sample - top depth'] * 0.3048)
    else:
        df_core.loc[index, 'Top depth'] = row['Core sample - top depth']

In [38]:
for index, row in df_core.iterrows():
    if row['Core sample depth - uom'] == '[ft  ]':
        df_core.loc[index, 'Base depth'] = (row['Core sample -  bottom depth'] * 0.3048)
    else:
        df_core.loc[index, 'Base depth'] = row['Core sample -  bottom depth']

In [39]:
df_core.isnull().sum()

Wellbore                         0
Core sample number               0
Core sample - top depth          0
Core sample -  bottom depth      0
Core sample depth - uom          0
Total core sample length [m]     0
Number of cores samples          1
Cores available for sampling?    0
NPDID wellbore                   0
Date updated                     0
Date sync NPD                    0
Top depth                        0
Base depth                       0
dtype: int64

In [40]:
df_core.dtypes
# Note extra space in 'Core sample -  bottom depth'

Wellbore                                 object
Core sample number                        int64
Core sample - top depth                 float64
Core sample -  bottom depth             float64
Core sample depth - uom                  object
Total core sample length [m]            float64
Number of cores samples                 float64
Cores available for sampling?            object
NPDID wellbore                            int64
Date updated                     datetime64[ns]
Date sync NPD                            object
Top depth                               float64
Base depth                              float64
dtype: object

In [41]:
df_core = df_core[['Wellbore', 'NPDID wellbore', 'Top depth', 'Base depth', 'Core sample number', ]].round(2)
df_core

,Wellbore,NPDID wellbore,Top depth,Base depth,Core sample number
0,1/2-1,1382,3111.40,3111.52,1
1,1/2-1,1382,3113.84,3119.02,2
3,1/2-1,1382,3126.03,3135.17,4
4,1/2-1,1382,3135.17,3157.12,5
5,1/2-1,1382,3157.12,3159.13,6
...,...,...,...,...,...
8273,9/2-A-4,2828,4648.50,4676.70,2
8274,9/2-A-4,2828,4676.70,4704.36,3
8275,9/3-1,921,1798.00,1805.40,1
8276,9/8-1,145,1926.00,1933.35,1


In [42]:
# Rename columns
rename_cols = {'Wellbore' : 'Well',
               'Core sample number': 'Legend'
               }
    
# Apply renaming
df_core.rename(columns=rename_cols, inplace=True)
df_core.head(3)

,Well,NPDID wellbore,Top depth,Base depth,Legend
0,1/2-1,1382,3111.40,3111.52,1
1,1/2-1,1382,3113.84,3119.02,2
3,1/2-1,1382,3126.03,3135.17,4


In [43]:
# Output file
output_to_csv(file='wellbore_core', df=df_core)

Saved to: output data/wellbore_core.csv


,Well,NPDID wellbore,Top depth,Base depth,Legend
0,1/2-1,1382,3111.40,3111.52,1
1,1/2-1,1382,3113.84,3119.02,2
2,1/2-1,1382,3126.03,3135.17,4


<h3>Core Photos</h3>

In [44]:
if data_source == 'web':
    df_core_photo = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_core_photo&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.129.189&CultureCode=en')

if data_source == 'file':
    df_core_photo = pd.read_excel('input data/wellbore_core_photo.xlsx')

df_core_photo.head(3)

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
0,1/2-1,1,10208-10228ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25
1,1/2-1,2,19228-10262ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25
2,1/2-1,3,10262-10277ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25


In [45]:
# See https://pythex.org/

# Match pattern:
# 10208-10228ft
# 1802-1805m

pat = '\d{3,5}-\d{3,5}\D{1,2}'
    
#filt = df_core_photo['Core photo title'].str.extract(pat)
filt = df_core_photo['Core photo title'].str.contains(pat)

# Check rows that match pattern
df_core_photo[filt].head(3)

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
0,1/2-1,1,10208-10228ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25
1,1/2-1,2,19228-10262ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25
2,1/2-1,3,10262-10277ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25


In [46]:
# Check rows that do not match pattern and make corrections
df_core_photo[~filt]

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
1586,16/1-5,6,2044,https://factpages.npd.no/pbl/core_photo_jpgs/3279_06_2044_2049m.jpg,3279,2019-04-25
2121,2/4-X-47,1,15736-15751,https://factpages.npd.no/pbl/core_photo_jpgs/3157_01_15736-15751.jpg,3157,2019-04-25
2122,2/4-X-47,2,15751-15766,https://factpages.npd.no/pbl/core_photo_jpgs/3157_02_15751-15766.jpg,3157,2019-04-25
2123,2/4-X-47,3,15766-15781,https://factpages.npd.no/pbl/core_photo_jpgs/3157_03_15766-15781.jpg,3157,2019-04-25
2124,2/4-X-47,4,15781-15796,https://factpages.npd.no/pbl/core_photo_jpgs/3157_04_15781-15796.jpg,3157,2019-04-25
...,...,...,...,...,...,...
18928,6609/7-1,7,1944,https://factpages.npd.no/pbl/core_photo_jpgs/19_07_1944_1945m.jpg,19,2019-04-25
19559,7119/12-2,11,1408,https://factpages.npd.no/pbl/core_photo_jpgs/114_11_1408_1411m.jpg,114,2019-04-25
19977,7120/8-1,26,2233-2235,https://factpages.npd.no/pbl/core_photo_jpgs/120_26_2233-2235.jpg,120,2019-04-25
20016,7120/8-2,26,2201,https://factpages.npd.no/pbl/core_photo_jpgs/50_26_2201_2206m.jpg,50,2019-04-25


In [47]:
# Note 95 rows with erroneous values
# Apply obvious corrections then drop the rest.

# Values for well 2/4-X-47 are obviously in ft.
filt_correction = df_core_photo['Wellbore'] == '2/4-X-47'
df_core_photo.loc[filt_correction, 'Core photo title'] = (df_core_photo['Core photo title'] + 'm')
df_core_photo.loc[filt_correction]

# There are other obvious corrections to be made, but leave for now.
# Example below, but don't do this on .loc as index may as more wells added.

#['Core photo title'].loc[14234] = '2482-2483m'

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
2121,2/4-X-47,1,15736-15751m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_01_15736-15751.jpg,3157,2019-04-25
2122,2/4-X-47,2,15751-15766m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_02_15751-15766.jpg,3157,2019-04-25
2123,2/4-X-47,3,15766-15781m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_03_15766-15781.jpg,3157,2019-04-25
2124,2/4-X-47,4,15781-15796m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_04_15781-15796.jpg,3157,2019-04-25
2125,2/4-X-47,5,15796-15811m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_05_15796-15811.jpg,3157,2019-04-25
2126,2/4-X-47,6,15811-15826m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_06_15811-15826.jpg,3157,2019-04-25
2127,2/4-X-47,8,15826-15837m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_08_15826-15837.jpg,3157,2019-04-25
2128,2/4-X-47,9,15837-15852m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_09_15837-15852.jpg,3157,2019-04-25
2129,2/4-X-47,10,15852-15865m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_10_15852-15865.jpg,3157,2019-04-25


In [48]:
# Assign rows that do not match pattern to new dataframe

df_core_photo_deletedrows = df_core_photo[~filt]
df_core_photo_deletedrows

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
1586,16/1-5,6,2044,https://factpages.npd.no/pbl/core_photo_jpgs/3279_06_2044_2049m.jpg,3279,2019-04-25
2121,2/4-X-47,1,15736-15751m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_01_15736-15751.jpg,3157,2019-04-25
2122,2/4-X-47,2,15751-15766m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_02_15751-15766.jpg,3157,2019-04-25
2123,2/4-X-47,3,15766-15781m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_03_15766-15781.jpg,3157,2019-04-25
2124,2/4-X-47,4,15781-15796m,https://factpages.npd.no/pbl/core_photo_jpgs/3157_04_15781-15796.jpg,3157,2019-04-25
...,...,...,...,...,...,...
18928,6609/7-1,7,1944,https://factpages.npd.no/pbl/core_photo_jpgs/19_07_1944_1945m.jpg,19,2019-04-25
19559,7119/12-2,11,1408,https://factpages.npd.no/pbl/core_photo_jpgs/114_11_1408_1411m.jpg,114,2019-04-25
19977,7120/8-1,26,2233-2235,https://factpages.npd.no/pbl/core_photo_jpgs/120_26_2233-2235.jpg,120,2019-04-25
20016,7120/8-2,26,2201,https://factpages.npd.no/pbl/core_photo_jpgs/50_26_2201_2206m.jpg,50,2019-04-25


In [49]:
# Keep only rows that do match pattern
# Dumps the rest (e.g. '2044', 'Core 2')

df_core_photo = df_core_photo[filt]
print(df_core_photo.shape)
df_core_photo

(20849, 6)


,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
0,1/2-1,1,10208-10228ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25
1,1/2-1,2,19228-10262ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25
2,1/2-1,3,10262-10277ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25
3,1/2-1,4,10277-10292ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_04_10277-10292ft.jpg,1382,2019-04-25
4,1/2-1,5,10292-10307ft,https://factpages.npd.no/pbl/core_photo_jpgs/1382_05_10292-10307ft.jpg,1382,2019-04-25
...,...,...,...,...,...,...
20939,9/2-A-4,13,4691-4696m,https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25
20940,9/2-A-4,14,4696-4701m,https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25
20941,9/2-A-4,15,4701-4704m,https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25
20942,9/3-1,1,1798-1802m,https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25


In [50]:
# Check for null values
df_core_photo.isna().sum()

Wellbore              0
Core sample number    0
Core photo title      0
Core photo URL        0
NPDID wellbore        0
Date updated          0
dtype: int64

In [51]:
# Check datatypes
df_core_photo.dtypes

Wellbore                      object
Core sample number             int64
Core photo title              object
Core photo URL                object
NPDID wellbore                 int64
Date updated          datetime64[ns]
dtype: object

In [52]:
df_core_photo['Core photo title'].replace({'mj': 'm', #one erroneous 'mj' value
                                           'n': ',m', #one erroneous 'n' value
                                           'm': ',m', #then replace all 'm'
                                           'M': ',m',
                                           'ft': ',ft',
                                           'FT': ',ft'
                                          }, regex=True, inplace=True)

df_core_photo['Core photo title'].replace({'-': ','}, regex=True, inplace=True)

df_core_photo.head(3)

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25


In [53]:
df_core_photo.tail()

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated
20939,9/2-A-4,13,"4691,4696,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25
20940,9/2-A-4,14,"4696,4701,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25
20941,9/2-A-4,15,"4701,4704,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25
20942,9/3-1,1,"1798,1802,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25
20943,9/3-1,2,"1802,1805,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_02_1802-1805m.jpg,921,2019-04-25


In [54]:
df_core_photo[['Top depth', 'Base depth', 'Unit']] = df_core_photo['Core photo title'].str.split(pat=',', n=2, expand=True)
df_core_photo

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated,Top depth,Base depth,Unit
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25,10208,10228,ft
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25,19228,10262,ft
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25,10262,10277,ft
3,1/2-1,4,"10277,10292,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_04_10277-10292ft.jpg,1382,2019-04-25,10277,10292,ft
4,1/2-1,5,"10292,10307,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_05_10292-10307ft.jpg,1382,2019-04-25,10292,10307,ft
...,...,...,...,...,...,...,...,...,...
20939,9/2-A-4,13,"4691,4696,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25,4691,4696,m
20940,9/2-A-4,14,"4696,4701,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25,4696,4701,m
20941,9/2-A-4,15,"4701,4704,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25,4701,4704,m
20942,9/3-1,1,"1798,1802,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25,1798,1802,m


In [55]:
df_core_photo['Unit'].unique()

array(['ft', 'm'], dtype=object)

In [56]:
# Drop columns that contain nulls
df_core_photo.isna().sum()

Wellbore              0
Core sample number    0
Core photo title      0
Core photo URL        0
NPDID wellbore        0
Date updated          0
Top depth             0
Base depth            0
Unit                  0
dtype: int64

In [57]:
df_core_photo

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated,Top depth,Base depth,Unit
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25,10208,10228,ft
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25,19228,10262,ft
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25,10262,10277,ft
3,1/2-1,4,"10277,10292,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_04_10277-10292ft.jpg,1382,2019-04-25,10277,10292,ft
4,1/2-1,5,"10292,10307,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_05_10292-10307ft.jpg,1382,2019-04-25,10292,10307,ft
...,...,...,...,...,...,...,...,...,...
20939,9/2-A-4,13,"4691,4696,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25,4691,4696,m
20940,9/2-A-4,14,"4696,4701,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25,4696,4701,m
20941,9/2-A-4,15,"4701,4704,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25,4701,4704,m
20942,9/3-1,1,"1798,1802,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25,1798,1802,m


In [58]:
df_core_photo.dtypes

Wellbore                      object
Core sample number             int64
Core photo title              object
Core photo URL                object
NPDID wellbore                 int64
Date updated          datetime64[ns]
Top depth                     object
Base depth                    object
Unit                          object
dtype: object

In [59]:
# Convert all top depths to metres

for index, row in df_core_photo.iterrows():
    if row['Unit'] == 'ft':
        df_core_photo.loc[index, 'Top depth'] = int(row['Top depth']) * 0.3048
    else:
        df_core_photo.loc[index, 'Top depth'] = int(row['Top depth'])

In [60]:
# Convert all base depths to metres

for index, row in df_core_photo.iterrows():
    if row['Unit'] == 'ft':
        df_core_photo.loc[index, 'Base depth'] = int(row['Base depth']) * 0.3048
    else:
        df_core_photo.loc[index, 'Base depth'] = int(row['Base depth'])

In [61]:
df_core_photo.round(2).head(3)

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated,Top depth,Base depth,Unit
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25,3111.4,3117.49,ft
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25,5860.69,3127.86,ft
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25,3127.86,3132.43,ft


In [62]:
# Add a new column with the filepath, e.g. '.\3279_06_2044_2049m.jpg'
# Where . represents the current directory

# Escaped insert backslash
df_core_photo['Legend'] = '.\\' + df_core_photo['Core photo URL'].str.split('/').str[-1]
df_core_photo

,Wellbore,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated,Top depth,Base depth,Unit,Legend
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25,3111.4,3117.49,ft,.\1382_01_10208-10228ft.jpg
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25,5860.69,3127.86,ft,.\1382_02_19228-10262ft.jpg
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25,3127.86,3132.43,ft,.\1382_03_10262-10277ft.jpg
3,1/2-1,4,"10277,10292,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_04_10277-10292ft.jpg,1382,2019-04-25,3132.43,3137,ft,.\1382_04_10277-10292ft.jpg
4,1/2-1,5,"10292,10307,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_05_10292-10307ft.jpg,1382,2019-04-25,3137,3141.57,ft,.\1382_05_10292-10307ft.jpg
...,...,...,...,...,...,...,...,...,...,...
20939,9/2-A-4,13,"4691,4696,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25,4691,4696,m,.\2828_13_4691-4696m.jpg
20940,9/2-A-4,14,"4696,4701,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25,4696,4701,m,.\2828_14_4696-4701m.jpg
20941,9/2-A-4,15,"4701,4704,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25,4701,4704,m,.\2828_15_4701-4704m.jpg
20942,9/3-1,1,"1798,1802,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25,1798,1802,m,.\921_01_1798-1802m.jpg


In [63]:
# Rename columns
rename_cols = {'Wellbore' : 'Well'}

# Apply renaming
df_core_photo.rename(columns=rename_cols, inplace=True)
df_core_photo

,Well,Core sample number,Core photo title,Core photo URL,NPDID wellbore,Date updated,Top depth,Base depth,Unit,Legend
0,1/2-1,1,"10208,10228,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_01_10208-10228ft.jpg,1382,2019-04-25,3111.4,3117.49,ft,.\1382_01_10208-10228ft.jpg
1,1/2-1,2,"19228,10262,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_02_19228-10262ft.jpg,1382,2019-04-25,5860.69,3127.86,ft,.\1382_02_19228-10262ft.jpg
2,1/2-1,3,"10262,10277,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_03_10262-10277ft.jpg,1382,2019-04-25,3127.86,3132.43,ft,.\1382_03_10262-10277ft.jpg
3,1/2-1,4,"10277,10292,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_04_10277-10292ft.jpg,1382,2019-04-25,3132.43,3137,ft,.\1382_04_10277-10292ft.jpg
4,1/2-1,5,"10292,10307,ft",https://factpages.npd.no/pbl/core_photo_jpgs/1382_05_10292-10307ft.jpg,1382,2019-04-25,3137,3141.57,ft,.\1382_05_10292-10307ft.jpg
...,...,...,...,...,...,...,...,...,...,...
20939,9/2-A-4,13,"4691,4696,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_13_4691-4696m.jpg,2828,2019-04-25,4691,4696,m,.\2828_13_4691-4696m.jpg
20940,9/2-A-4,14,"4696,4701,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_14_4696-4701m.jpg,2828,2019-04-25,4696,4701,m,.\2828_14_4696-4701m.jpg
20941,9/2-A-4,15,"4701,4704,m",https://factpages.npd.no/pbl/core_photo_jpgs/2828_15_4701-4704m.jpg,2828,2019-04-25,4701,4704,m,.\2828_15_4701-4704m.jpg
20942,9/3-1,1,"1798,1802,m",https://factpages.npd.no/pbl/core_photo_jpgs/921_01_1798-1802m.jpg,921,2019-04-25,1798,1802,m,.\921_01_1798-1802m.jpg


In [64]:
df_core_photo = df_core_photo[['Well', 'Top depth', 'Base depth', 'Legend']]
df_core_photo.head(3)

,Well,Top depth,Base depth,Legend
0,1/2-1,3111.4,3117.49,.\1382_01_10208-10228ft.jpg
1,1/2-1,5860.69,3127.86,.\1382_02_19228-10262ft.jpg
2,1/2-1,3127.86,3132.43,.\1382_03_10262-10277ft.jpg


In [65]:
# Output file
output_to_csv(file='wellbore_core_photo', df=df_core_photo)

Saved to: output data/wellbore_core_photo.csv


,Well,Top depth,Base depth,Legend
0,1/2-1,3111.3984,3117.4944,.\1382_01_10208-10228ft.jpg
1,1/2-1,5860.6944,3127.8576,.\1382_02_19228-10262ft.jpg
2,1/2-1,3127.8576,3132.4296,.\1382_03_10262-10277ft.jpg


<h3>Thin Sections</h3>

In [66]:
if data_source == 'web':
    df_thin_section = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_thin_section&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')

if data_source == 'file':
    df_thin_section = pd.read_excel('input data/wellbore_thin_section.xlsx')

In [67]:
print(df_thin_section.shape)
df_thin_section.head(3)

(2136, 7)


,Wellbore,Number,Depth,Unit,NPDID wellbore,Date updated,Date sync NPD
0,1/3-1,117,11221.0,[ft ],154,2019-04-25,16.02.2020
1,1/3-1,118,11221.0,[ft ],154,2019-04-25,16.02.2020
2,1/3-5,121,4809.0,[m ],223,2018-12-20,16.02.2020


In [68]:
df_thin_section['Unit'].unique()

array(['[ft  ]', '[m   ]'], dtype=object)

In [69]:
df_thin_section.isna().sum()

Wellbore          0
Number            0
Depth             0
Unit              0
NPDID wellbore    0
Date updated      0
Date sync NPD     0
dtype: int64

In [70]:
for index, row in df_thin_section.iterrows():
    if row['Unit'] == '[ft  ]':
        df_thin_section.loc[index, 'Depth'] = row['Depth'] * 0.3048
    else:
        df_thin_section.loc[index, 'Depth'] = row['Depth']
        
df_thin_section.drop(columns='Unit', inplace=True)

In [71]:
df_thin_section.head(3)

,Wellbore,Number,Depth,NPDID wellbore,Date updated,Date sync NPD
0,1/3-1,117,3420.1608,154,2019-04-25,16.02.2020
1,1/3-1,118,3420.1608,154,2019-04-25,16.02.2020
2,1/3-5,121,4809.0000,223,2018-12-20,16.02.2020


In [72]:
for index, row in df_thin_section.iterrows():
    df_thin_section.loc[index, 'Legend'] = 'Thin section no. ' + str(row['Number'])
    
df_thin_section.head(3)

,Wellbore,Number,Depth,NPDID wellbore,Date updated,Date sync NPD,Legend
0,1/3-1,117,3420.1608,154,2019-04-25,16.02.2020,Thin section no. 117
1,1/3-1,118,3420.1608,154,2019-04-25,16.02.2020,Thin section no. 118
2,1/3-5,121,4809.0000,223,2018-12-20,16.02.2020,Thin section no. 121


In [73]:
# Rename columns
rename_cols = {'Wellbore' : 'Well'
               }
    
# Apply renaming
df_thin_section.rename(columns=rename_cols, inplace=True)
df_thin_section

,Well,Number,Depth,NPDID wellbore,Date updated,Date sync NPD,Legend
0,1/3-1,117,3420.1608,154,2019-04-25,16.02.2020,Thin section no. 117
1,1/3-1,118,3420.1608,154,2019-04-25,16.02.2020,Thin section no. 118
2,1/3-5,121,4809.0000,223,2018-12-20,16.02.2020,Thin section no. 121
3,1/3-5,122,4812.0000,223,2018-12-20,16.02.2020,Thin section no. 122
4,1/3-5,2319,4805.3500,223,2019-03-27,16.02.2020,Thin section no. 2319
...,...,...,...,...,...,...,...
2131,8/3-1,790,3010.0000,142,2018-12-20,16.02.2020,Thin section no. 790
2132,9/4-3,793,3820.0000,152,2018-12-20,16.02.2020,Thin section no. 793
2133,9/4-3,794,3840.0000,152,2018-12-20,16.02.2020,Thin section no. 794
2134,9/4-3,795,3860.0000,152,2018-12-20,16.02.2020,Thin section no. 795


In [74]:
df_thin_section = df_thin_section[['Well', 'Depth', 'Legend']]
df_thin_section = df_thin_section.round(2)
df_thin_section

,Well,Depth,Legend
0,1/3-1,3420.16,Thin section no. 117
1,1/3-1,3420.16,Thin section no. 118
2,1/3-5,4809.00,Thin section no. 121
3,1/3-5,4812.00,Thin section no. 122
4,1/3-5,4805.35,Thin section no. 2319
...,...,...,...
2131,8/3-1,3010.00,Thin section no. 790
2132,9/4-3,3820.00,Thin section no. 793
2133,9/4-3,3840.00,Thin section no. 794
2134,9/4-3,3860.00,Thin section no. 795


In [75]:
# Output file
output_to_csv(file='wellbore_thin_section', df=df_thin_section)

Saved to: output data/wellbore_thin_section.csv


,Well,Depth,Legend
0,1/3-1,3420.16,Thin section no. 117
1,1/3-1,3420.16,Thin section no. 118
2,1/3-5,4809.00,Thin section no. 121


In [76]:
# Point - comment
# No new IC data types

<h3>CO2</h3>

In [77]:
if data_source == 'web':
    df_co2 = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_co2&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en',
                     skiprows=[0])

if data_source == 'file':
    df_co2 = pd.read_excel('input data/wellbore_co2.xlsx', sheet_name='wellbore_co2', 
                           skiprows=[0])

In [78]:
print(df_co2.shape)
df_co2.head(3)

(94, 10)


,Unnamed: 0,Wellbore name,Sample sequence number,Sample top depth [m],Sample bottom depth [m],CO2 [vol %],Sample method,Sample type,NPDID wellbore,Date sync NPD
0,NaN,15/3-7,1,4073.8,4610.1,18900,NaN,MDT,4055,16.02.2020
1,NaN,17/3-1,1,2406.1,2406.1,3020,NaN,MDT,2576,16.02.2020
2,NaN,2/7-22,1,4489.0,4496.0,4500,NaN,DST,1495,16.02.2020


In [79]:
df_co2.drop(labels='Unnamed: 0', axis=1, inplace=True)
df_co2

,Wellbore name,Sample sequence number,Sample top depth [m],Sample bottom depth [m],CO2 [vol %],Sample method,Sample type,NPDID wellbore,Date sync NPD
0,15/3-7,1,4073.8,4610.1,18900,NaN,MDT,4055,16.02.2020
1,17/3-1,1,2406.1,2406.1,3020,NaN,MDT,2576,16.02.2020
2,2/7-22,1,4489.0,4496.0,4500,NaN,DST,1495,16.02.2020
3,3/7-4,1,3440.0,3537.0,4620,NaN,DST,1467,16.02.2020
4,30/3-4,1,3079.0,3096.0,5300,NaN,DST,460,16.02.2020
...,...,...,...,...,...,...,...,...,...
89,7120/8-1,1,2093.0,2110.0,6100,NaN,DST,120,16.02.2020
90,7121/4-1,1,2465.9,2471.9,10200,NaN,DST,135,16.02.2020
91,7121/7-1,1,1867.0,1872.0,6100,NaN,DST,130,16.02.2020
92,7121/7-2,1,1881.8,1889.8,8600,NaN,DST,881,16.02.2020


In [80]:
df_co2.isna().sum()

Wellbore name               0
Sample sequence number      0
Sample top depth [m]        0
Sample bottom depth [m]     0
CO2 [vol %]                 0
Sample method              94
Sample type                 0
NPDID wellbore              0
Date sync NPD               0
dtype: int64

In [81]:
df_co2.drop(labels=['Sample method', 'NPDID wellbore', 'Date sync NPD'], axis=1, inplace=True)

In [82]:
df_co2.columns

Index(['Wellbore name', 'Sample sequence number', 'Sample top depth [m]',
       'Sample bottom depth [m]', 'CO2 [vol %]', 'Sample type'],
      dtype='object')

In [83]:
# Rename columns
rename_cols = {'Wellbore name' : 'Well',
               'Sample top depth [m]' : 'Top depth',
               'Sample bottom depth [m]' : 'Base depth'
               }
    
# Apply renaming
df_co2.rename(columns=rename_cols, inplace=True)
df_co2

,Well,Sample sequence number,Top depth,Base depth,CO2 [vol %],Sample type
0,15/3-7,1,4073.8,4610.1,18900,MDT
1,17/3-1,1,2406.1,2406.1,3020,MDT
2,2/7-22,1,4489.0,4496.0,4500,DST
3,3/7-4,1,3440.0,3537.0,4620,DST
4,30/3-4,1,3079.0,3096.0,5300,DST
...,...,...,...,...,...,...
89,7120/8-1,1,2093.0,2110.0,6100,DST
90,7121/4-1,1,2465.9,2471.9,10200,DST
91,7121/7-1,1,1867.0,1872.0,6100,DST
92,7121/7-2,1,1881.8,1889.8,8600,DST


In [84]:
# Output file
output_to_csv(file='wellbore_co2', df=df_co2)

Saved to: output data/wellbore_co2.csv


,Well,Sample sequence number,Top depth,Base depth,CO2 [vol %],Sample type
0,15/3-7,1,4073.8,4610.1,18900,MDT
1,17/3-1,1,2406.1,2406.1,3020,MDT
2,2/7-22,1,4489.0,4496.0,4500,DST


In [85]:
# Interval-point
# Create data types: 'Sample sequence number', 'CO2 [vol %]', 'Sample type'

<h3>Oil Samples</h3>

In [86]:
if data_source == 'web':
    df_oil_sample = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_oil_sample&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')

if data_source == 'file':
    df_oil_sample = pd.read_excel('input data/wellbore_oil_sample.xlsx')

In [87]:
print(df_oil_sample.shape)
df_oil_sample.head(3)

(1005, 11)


,Wellbore,Test type,Bottle number,Top depth MD [m],Bottom depth MD [m],Fluid type,Test time,Received date,NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,DST,DST1,3122.3,3137.0,,1989-05-27 06:00:00,1991-09-09,1382,2016-05-19,16.02.2020
1,1/3-10,DST,,0.0,0.0,,NaT,2011-08-06,5614,2016-05-19,16.02.2020
2,1/3-11,DST,,0.0,0.0,,NaT,NaT,5806,2016-05-19,16.02.2020


In [88]:
df_oil_sample.isna().sum()

Wellbore                 0
Test type                0
Bottle number            0
Top depth MD [m]         0
Bottom depth MD [m]      0
Fluid type               0
Test time              196
Received date           62
NPDID wellbore           0
Date updated             0
Date sync NPD            0
dtype: int64

In [89]:
df_oil_sample.drop(labels=['NPDID wellbore', 'Date updated', 'Date sync NPD'], axis=1, inplace=True)

In [90]:
df_oil_sample.columns

Index(['Wellbore', 'Test type', 'Bottle number', 'Top depth MD [m]',
       'Bottom depth MD [m]', 'Fluid type', 'Test time ', 'Received date'],
      dtype='object')

In [91]:
# Rename columns
rename_cols = {'Wellbore' : 'Well',
               'Top depth MD [m]' : 'Top depth',
               'Bottom depth MD [m]' : 'Base depth'
               }
    
# Apply renaming
df_oil_sample.rename(columns=rename_cols, inplace=True)
df_oil_sample

,Well,Test type,Bottle number,Top depth,Base depth,Fluid type,Test time,Received date
0,1/2-1,DST,DST1,3122.30,3137.00,,1989-05-27 06:00:00,1991-09-09
1,1/3-10,DST,,0.00,0.00,,NaT,2011-08-06
2,1/3-11,DST,,0.00,0.00,,NaT,NaT
3,1/3-11,MDT,,3294.50,0.00,OIL,NaT,2014-04-03
4,1/3-3,DST,DST3A,4202.00,4208.00,WATER,1983-03-06 00:00:00,1991-10-28
...,...,...,...,...,...,...,...,...
1000,7324/7-3 S,DST,,2232.49,1771.03,OIL,NaT,2016-08-11
1001,7324/8-1,MDT,,664.00,0.00,OIL,NaT,2015-07-20
1002,7324/8-1,MDT,,678.00,0.00,OIL,NaT,2015-07-20
1003,9/2-1,DST,TEST3,3177.00,3210.00,,1987-04-20 00:00:00,1991-10-01


In [92]:
df_oil_sample

,Well,Test type,Bottle number,Top depth,Base depth,Fluid type,Test time,Received date
0,1/2-1,DST,DST1,3122.30,3137.00,,1989-05-27 06:00:00,1991-09-09
1,1/3-10,DST,,0.00,0.00,,NaT,2011-08-06
2,1/3-11,DST,,0.00,0.00,,NaT,NaT
3,1/3-11,MDT,,3294.50,0.00,OIL,NaT,2014-04-03
4,1/3-3,DST,DST3A,4202.00,4208.00,WATER,1983-03-06 00:00:00,1991-10-28
...,...,...,...,...,...,...,...,...
1000,7324/7-3 S,DST,,2232.49,1771.03,OIL,NaT,2016-08-11
1001,7324/8-1,MDT,,664.00,0.00,OIL,NaT,2015-07-20
1002,7324/8-1,MDT,,678.00,0.00,OIL,NaT,2015-07-20
1003,9/2-1,DST,TEST3,3177.00,3210.00,,1987-04-20 00:00:00,1991-10-01


In [93]:
# Output file
output_to_csv(file='wellbore_oil_sample', df=df_oil_sample)

Saved to: output data/wellbore_oil_sample.csv


,Well,Test type,Bottle number,Top depth,Base depth,Fluid type,Test time,Received date
0,1/2-1,DST,DST1,3122.3,3137.0,,1989-05-27 06:00:00,1991-09-09 00:00:00
1,1/3-10,DST,,0.0,0.0,,NaN,2011-08-06 00:00:00
2,1/3-11,DST,,0.0,0.0,,NaN,NaN


In [94]:
df_oil_sample.columns

# Interval - point
# Create data types: 'Test type', 'Bottle number', 'Fluid type', 'Test time ', 'Received date'

# What to do about rows with only with 0/Nan values?

Index(['Well', 'Test type', 'Bottle number', 'Top depth', 'Base depth',
       'Fluid type', 'Test time ', 'Received date'],
      dtype='object')

<h3>Lithostratigraphy</h3>

In [95]:
# Lithostrat available in two places. 
# Compare the length, and number of unique wells in both sources.

# (A) NPD FactPages > Wellbore > Table View > With > Lithostratigraphy
    # File: wellbore_formation_top.xlsx
    # Sheet: wellbore_formation_top
    # Link: https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_formation_top&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en

df_a = pd.read_excel('input data/wellbore_formation_top.xlsx', sheet_name='wellbore_formation_top')
print('Source A:', df_a.shape)
print(df_a['Wellbore name'].nunique())

# (B) NPD FactPages > Stratigraphy > Table View > Wellbores
    # File: strat_litho_wellbore.xlsx
    # Sheet: strat_litho_wellbore
    # Link: https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/strat_litho_wellbore&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en

df_b = pd.read_excel('input data/strat_litho_wellbore.xlsx', sheet_name='strat_litho_wellbore')
print('Source B:', df_b.shape)
print(df_b['Wellbore name'].nunique())

# Both contain the same number of rows.
# Source A is preferrable as it has an exra column, 'Lithostrat. unit, parent'
# which will come in handy assigning parents to each text dictionary entry.

Source A: (36035, 11)
1777
Source B: (36037, 10)
1778


In [96]:
# Use Source A

if data_source == 'web':
    df_lithostrat = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_formation_top&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')
    
if data_source == 'file':
    df_lithostrat = pd.read_excel('input data/wellbore_formation_top.xlsx')

# Print column titles
print("Lithostratigraphy wellbore well header column titles:")
print(list(df_lithostrat.columns))

Lithostratigraphy wellbore well header column titles:
['Wellbore name', 'Top depth [m]', 'Bottom depth [m]', 'Lithostrat. unit', 'Level', 'Lithostrat. unit, parent', 'NPDID wellbore', 'NPDID lithostrat. unit', 'NPDID parent lithostrat. unit', 'Date updated', 'Date sync NPD']


In [97]:
(num_lithostrat_rows, num_lithostrat_cols) = df_lithostrat.shape
print('{} rows and {} columns in Exploration wells.'.format(num_lithostrat_rows, num_lithostrat_cols))

36035 rows and 11 columns in Exploration wells.


In [98]:
df_lithostrat.head(3)

,Wellbore name,Top depth [m],Bottom depth [m],Lithostrat. unit,Level,"Lithostrat. unit, parent",NPDID wellbore,NPDID lithostrat. unit,NPDID parent lithostrat. unit,Date updated,Date sync NPD
0,9/8-1,1597.0,1625.0,BLODØKS FM,FORMATION,SHETLAND GP,145,13,143.0,2019-10-03,16.02.2020
1,9/8-1,1922.0,2109.0,SANDNES FM,FORMATION,VESTLAND GP,145,139,186.0,2019-10-03,16.02.2020
2,9/8-1,1777.0,1922.0,BOKNFJORD GP,GROUP,None,145,15,NaN,2019-10-03,16.02.2020


In [99]:
df_lithostrat.tail()

,Wellbore name,Top depth [m],Bottom depth [m],Lithostrat. unit,Level,"Lithostrat. unit, parent",NPDID wellbore,NPDID lithostrat. unit,NPDID parent lithostrat. unit,Date updated,Date sync NPD
36030,1/2-1,94.0,1777.0,NORDLAND GP,GROUP,None,1382,113,NaN,2019-10-03,16.02.2020
36031,1/2-1,3514.0,3574.0,TOR FM,FORMATION,SHETLAND GP,1382,171,143.0,2019-10-03,16.02.2020
36032,1/2-1,3335.0,3407.0,MAUREEN FM,FORMATION,ROGALAND GP,1382,102,131.0,2019-10-03,16.02.2020
36033,1/2-1,1777.0,3059.0,HORDALAND GP,GROUP,None,1382,67,NaN,2019-10-03,16.02.2020
36034,1/2-1,3275.0,3335.0,LISTA FM,FORMATION,ROGALAND GP,1382,95,131.0,2019-10-03,16.02.2020


In [100]:
#Rename columns for csv
rename_stratcols = {'Wellbore name' : 'Well',
                    'Top depth [m]' : 'Top depth',
                    'Bottom depth [m]' : 'Base depth',
                    'Lithostrat. unit' : 'Legend'
                    }

#Apply renaming to dataframe
df_lithostrat.rename(columns=rename_stratcols, inplace=True)

# Create new dataframe called "df_formation_top"
# Need to keep other columns df_lithostrat for later when writing to database

df_formation_top = df_lithostrat[['Well', 'Top depth', 'Base depth', 'Legend']]
df_formation_top.head(3)

,Well,Top depth,Base depth,Legend
0,9/8-1,1597.0,1625.0,BLODØKS FM
1,9/8-1,1922.0,2109.0,SANDNES FM
2,9/8-1,1777.0,1922.0,BOKNFJORD GP


In [101]:
# Output file
output_to_csv(file='wellbore_formation_top', df=df_formation_top)

Saved to: output data/wellbore_formation_top.csv


,Well,Top depth,Base depth,Legend
0,9/8-1,1597.0,1625.0,BLODØKS FM
1,9/8-1,1922.0,2109.0,SANDNES FM
2,9/8-1,1777.0,1922.0,BOKNFJORD GP


<h3>Drill stem tests</h3>

In [102]:
if data_source == 'web':
    df_dst = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_dst&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')

if data_source == 'file':
    df_dst = pd.read_excel('input data/wellbore_dst.xlsx')

In [103]:
print(df_dst.shape)
df_dst.head(20)

(1167, 17)


,Wellbore,Test number,From depth MD [m],To depth MD [m],Choke size [mm],Final shut-in pressure [MPa],Final flow pressure [MPa],Bottom hole pressure [MPa],Oil [Sm3/day],Gas [Sm3/day],Oil density [g/cm3],Gas grav. rel.air,GOR [m3/m3],Downhole temperature [°C],NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,1.0,3122.3,3137.0,25.4,0.0,0.0,0.0,859,57000,0.810,0.000,66,0,1382,2016-05-19,16.02.2020
1,1/3-1,1.0,4585.0,4602.0,12.5,80.0,61.0,0.0,0,1132,0.000,0.000,0,0,154,2016-05-19,16.02.2020
2,1/3-1,2.0,4565.0,4602.0,12.5,52.0,40.0,0.0,0,6626,0.000,0.000,0,0,154,2016-05-19,16.02.2020
3,1/3-1,3.0,3356.0,3361.0,12.5,70.0,50.0,0.0,0,28317,0.000,0.000,0,0,154,2016-05-19,16.02.2020
4,1/3-10,1.0,3158.0,3288.0,19.0,9.0,0.0,38.0,457,212453,0.791,0.855,465,0,5614,2016-05-19,16.02.2020
5,1/3-3,1.0,4529.0,4552.0,25.4,0.0,0.0,0.0,0,0,0.000,0.000,0,0,87,2016-05-19,16.02.2020
6,1/3-3,2.0,4233.0,4240.0,9.5,0.0,0.0,0.0,0,0,0.000,0.000,0,0,87,2016-05-19,16.02.2020
7,1/3-3,3.1,4202.0,4208.0,50.8,0.0,0.0,0.0,0,0,0.000,0.000,0,0,87,2016-05-19,16.02.2020
8,1/3-3,3.3,4211.0,4214.0,6.4,0.0,0.0,0.0,143,28000,0.829,0.820,196,165,87,2016-05-19,16.02.2020
9,1/3-6,1.0,2960.5,2977.0,17.5,0.0,3.0,0.0,78,93300,0.780,0.780,1196,107,1521,2016-05-19,16.02.2020


In [104]:
df_dst.isna().sum()

Wellbore                        0
Test number                     0
From depth MD [m]               0
To depth MD [m]                 0
Choke size [mm]                 0
Final shut-in pressure [MPa]    0
Final flow pressure [MPa]       0
Bottom hole pressure [MPa]      0
Oil [Sm3/day]                   0
Gas [Sm3/day]                   0
Oil density [g/cm3]             0
Gas grav. rel.air               0
GOR [m3/m3]                     0
Downhole temperature [°C]       0
NPDID wellbore                  0
Date updated                    0
Date sync NPD                   0
dtype: int64

In [105]:
df_dst.drop(labels=['NPDID wellbore', 'Date updated', 'Date sync NPD'], axis=1, inplace=True)

In [106]:
df_dst.columns

Index(['Wellbore', 'Test number', 'From depth MD [m]', 'To depth MD [m]',
       'Choke size [mm]', 'Final shut-in pressure [MPa]',
       'Final flow pressure [MPa]', 'Bottom hole pressure [MPa]',
       'Oil [Sm3/day]', 'Gas [Sm3/day]', 'Oil density [g/cm3]',
       'Gas grav. rel.air', 'GOR [m3/m3]', 'Downhole temperature [°C]'],
      dtype='object')

In [107]:
#Rename well header columns to match dbo.WELLS (does capitalisation matter?)
rename_cols = {'Wellbore' : 'Well',
               'From depth MD [m]' : 'Top depth',
               'To depth MD [m]' : 'Base depth'
               }
    
#Apply renaming
df_dst.rename(columns=rename_cols, inplace=True)
df_dst.head(20)

,Well,Test number,Top depth,Base depth,Choke size [mm],Final shut-in pressure [MPa],Final flow pressure [MPa],Bottom hole pressure [MPa],Oil [Sm3/day],Gas [Sm3/day],Oil density [g/cm3],Gas grav. rel.air,GOR [m3/m3],Downhole temperature [°C]
0,1/2-1,1.0,3122.3,3137.0,25.4,0.0,0.0,0.0,859,57000,0.810,0.000,66,0
1,1/3-1,1.0,4585.0,4602.0,12.5,80.0,61.0,0.0,0,1132,0.000,0.000,0,0
2,1/3-1,2.0,4565.0,4602.0,12.5,52.0,40.0,0.0,0,6626,0.000,0.000,0,0
3,1/3-1,3.0,3356.0,3361.0,12.5,70.0,50.0,0.0,0,28317,0.000,0.000,0,0
4,1/3-10,1.0,3158.0,3288.0,19.0,9.0,0.0,38.0,457,212453,0.791,0.855,465,0
5,1/3-3,1.0,4529.0,4552.0,25.4,0.0,0.0,0.0,0,0,0.000,0.000,0,0
6,1/3-3,2.0,4233.0,4240.0,9.5,0.0,0.0,0.0,0,0,0.000,0.000,0,0
7,1/3-3,3.1,4202.0,4208.0,50.8,0.0,0.0,0.0,0,0,0.000,0.000,0,0
8,1/3-3,3.3,4211.0,4214.0,6.4,0.0,0.0,0.0,143,28000,0.829,0.820,196,165
9,1/3-6,1.0,2960.5,2977.0,17.5,0.0,3.0,0.0,78,93300,0.780,0.780,1196,107


In [108]:
# Output file
output_to_csv(file='wellbore_dst', df=df_dst)

Saved to: output data/wellbore_dst.csv


,Well,Test number,Top depth,Base depth,Choke size [mm],Final shut-in pressure [MPa],Final flow pressure [MPa],Bottom hole pressure [MPa],Oil [Sm3/day],Gas [Sm3/day],Oil density [g/cm3],Gas grav. rel.air,GOR [m3/m3],Downhole temperature [°C]
0,1/2-1,1.0,3122.3,3137.0,25.4,0.0,0.0,0.0,859,57000,0.81,0.0,66,0
1,1/3-1,1.0,4585.0,4602.0,12.5,80.0,61.0,0.0,0,1132,0.00,0.0,0,0
2,1/3-1,2.0,4565.0,4602.0,12.5,52.0,40.0,0.0,0,6626,0.00,0.0,0,0


<h3>Casing and leak-off tests</h3>

In [109]:
if data_source == 'web':
    df_casinglot = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_casing_and_lot&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')

if data_source == 'file':
    df_casinglot = pd.read_excel('input data/wellbore_casing_and_lot.xlsx')

In [110]:
print(df_casinglot.shape)
df_casinglot.head(3)

(7561, 11)


,Wellbore,Casing type,Casing diam. [inch],Casing depth [m],Hole diam. [inch],Hole depth[m],LOT mud eqv. [g/cm3],Formation test type,NPDID wellbore,Date updated,Date sync NPD
0,6507/6-4 S,CONDUCTOR,30,30.0,36,0.0,0.0,LOT,6725,2017-04-11,16.02.2020
1,6406/9-3,CONDUCTOR,36,45.0,42,0.0,0.0,,7141,2016-10-10,16.02.2020
2,2/5-4,CONDUCTOR,30,45.0,36,46.0,0.0,LOT,259,2017-04-11,16.02.2020


In [111]:
df_casinglot.isna().sum()

Wellbore                0
Casing type             0
Casing diam. [inch]     0
Casing depth [m]        0
Hole diam. [inch]       0
Hole depth[m]           0
LOT mud eqv. [g/cm3]    0
Formation test type     1
NPDID wellbore          0
Date updated            0
Date sync NPD           0
dtype: int64

In [112]:
df_casinglot.drop(labels=['NPDID wellbore', 'Date updated', 'Date sync NPD'], axis=1, inplace=True)

In [113]:
df_casinglot.columns

Index(['Wellbore', 'Casing type', 'Casing diam. [inch]', 'Casing depth [m]',
       'Hole diam. [inch]', 'Hole depth[m]', 'LOT mud eqv. [g/cm3]',
       'Formation test type'],
      dtype='object')

In [114]:
# Rename columns
rename_cols = {'Wellbore' : 'Well',
               'Casing depth [m]' : 'Depth'
               }
    
# Apply renaming
df_casinglot.rename(columns=rename_cols, inplace=True)
df_casinglot

,Well,Casing type,Casing diam. [inch],Depth,Hole diam. [inch],Hole depth[m],LOT mud eqv. [g/cm3],Formation test type
0,6507/6-4 S,CONDUCTOR,30,30.0,36,0.0,0.0,LOT
1,6406/9-3,CONDUCTOR,36,45.0,42,0.0,0.0,
2,2/5-4,CONDUCTOR,30,45.0,36,46.0,0.0,LOT
3,7/9-1,CONDUCTOR,30,108.0,36,110.0,0.0,LOT
4,8/12-1,CONDUCTOR,30,114.0,36,114.0,0.0,LOT
...,...,...,...,...,...,...,...,...
7556,34/10-48 S,OPEN HOLE,,7393.0,8 1/2,7393.0,0.0,LOT
7557,34/10-45 S,LINER,7,7594.0,8 1/2,7594.0,0.0,LOT
7558,34/10-46 B,LINER,7,7725.0,8 1/2,7725.0,0.0,LOT
7559,34/10-55 S,OPEN HOLE,,7811.0,8 1/2,7811.0,0.0,


In [115]:
# Output file
output_to_csv(file='wellbore_casing_and_lot', df=df_casinglot)

Saved to: output data/wellbore_casing_and_lot.csv


,Well,Casing type,Casing diam. [inch],Depth,Hole diam. [inch],Hole depth[m],LOT mud eqv. [g/cm3],Formation test type
0,6507/6-4 S,CONDUCTOR,30,30.0,36,0.0,0.0,LOT
1,6406/9-3,CONDUCTOR,36,45.0,42,0.0,0.0,
2,2/5-4,CONDUCTOR,30,45.0,36,46.0,0.0,LOT


<h3>Drilling mud</h3>

In [116]:
if data_source == 'web':
    df_mud = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_mud&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.189&CultureCode=en')
    
if data_source == 'file':
    df_mud = pd.read_excel('input data/wellbore_mud.xlsx')

In [117]:
print(df_mud.shape)
df_mud.head(3)

(33843, 11)


,Unnamed: 0,Wellbore,Depth MD [m],Mud weight [g/cm3],Visc. [mPa.s],Yield point [Pa],Mud type,Date measured,NPDID wellbore,Date updated,Date sync NPD
0,NaN,1/2-1,171.0,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020
1,NaN,1/2-1,279.2,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020
2,NaN,1/2-1,545.6,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020


In [118]:
df_mud.drop(labels='Unnamed: 0', axis=1, inplace=True)
df_mud.head(3)

,Wellbore,Depth MD [m],Mud weight [g/cm3],Visc. [mPa.s],Yield point [Pa],Mud type,Date measured,NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,171.0,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020
1,1/2-1,279.2,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020
2,1/2-1,545.6,1.04,0.0,0.0,WATER BASED,1989-03-29,1382,2017-04-11,16.02.2020


In [119]:
df_mud.isna().sum()

Wellbore                  0
Depth MD [m]              0
Mud weight [g/cm3]        0
Visc. [mPa.s]             0
Yield point [Pa]          0
Mud type                  1
Date measured         15684
NPDID wellbore            0
Date updated              0
Date sync NPD             0
dtype: int64

In [120]:
df_mud.drop(labels=['NPDID wellbore', 'Date updated', 'Date sync NPD'], axis=1, inplace=True)

In [121]:
df_mud.columns

Index(['Wellbore', 'Depth MD [m]', 'Mud weight [g/cm3]', 'Visc. [mPa.s]',
       'Yield point [Pa]', 'Mud type', 'Date measured'],
      dtype='object')

In [122]:
# Rename columns
rename_cols = {'Wellbore' : 'Well',
               'Depth MD [m]' : 'Depth'
               }
    
# Apply renaming
df_mud.rename(columns=rename_cols, inplace=True)
df_mud

,Well,Depth,Mud weight [g/cm3],Visc. [mPa.s],Yield point [Pa],Mud type,Date measured
0,1/2-1,171.0,1.04,0.0,0.0,WATER BASED,1989-03-29
1,1/2-1,279.2,1.04,0.0,0.0,WATER BASED,1989-03-29
2,1/2-1,545.6,1.04,0.0,0.0,WATER BASED,1989-03-29
3,1/2-1,563.9,1.04,0.0,0.0,WATER BASED,1989-03-29
4,1/2-1,648.3,1.04,0.0,0.0,WATER BASED,1989-03-29
...,...,...,...,...,...,...,...
33838,9/4-5,5881.1,0.00,0.0,0.0,DUMMY,NaT
33839,9/8-1,137.0,0.00,0.0,0.0,seawater,NaT
33840,9/8-1,360.0,1.11,0.0,0.0,seawater,NaT
33841,9/8-1,1219.0,1.13,0.0,0.0,waterbased,NaT


In [123]:
# Output file
output_to_csv(file='wellbore_mud', df=df_mud)

Saved to: output data/wellbore_mud.csv


,Well,Depth,Mud weight [g/cm3],Visc. [mPa.s],Yield point [Pa],Mud type,Date measured
0,1/2-1,171.0,1.04,0.0,0.0,WATER BASED,1989-03-29
1,1/2-1,279.2,1.04,0.0,0.0,WATER BASED,1989-03-29
2,1/2-1,545.6,1.04,0.0,0.0,WATER BASED,1989-03-29


<h3>Documents</h3>

In [124]:
if data_source == 'web':
    df_document = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_document&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=108.171.128.188&CultureCode=en')
    
if data_source == 'file':
    df_document = pd.read_excel('input data/wellbore_document.xlsx')

df_document.head(3)

,Wellbore,Document type,Document name,Document URL,Document format,Document size [KB],NPDID wellbore,Date updated,Date sync NPD
0,1/2-1,GEOCHEMICAL INFORMATION,1382_1,https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf,pdf,174,1382,2019-04-25,16.02.2020
1,1/2-1,GEOCHEMICAL INFORMATION,1382_2,https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf,pdf,3408,1382,2019-04-25,16.02.2020
2,1/2-1,OLD NPD WDSS,1382_01_WDSS_General_Information,https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf,pdf,258,1382,2019-04-25,16.02.2020


In [125]:
df_document['Title'] = df_document['Wellbore'] + ' ' + df_document['Document type'] + ': ' + df_document['Document name'] + ' (' + df_document['Document format'] + ')'
df_document.head(3)

,Wellbore,Document type,Document name,Document URL,Document format,Document size [KB],NPDID wellbore,Date updated,Date sync NPD,Title
0,1/2-1,GEOCHEMICAL INFORMATION,1382_1,https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf,pdf,174,1382,2019-04-25,16.02.2020,1/2-1 GEOCHEMICAL INFORMATION: 1382_1 (pdf)
1,1/2-1,GEOCHEMICAL INFORMATION,1382_2,https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf,pdf,3408,1382,2019-04-25,16.02.2020,1/2-1 GEOCHEMICAL INFORMATION: 1382_2 (pdf)
2,1/2-1,OLD NPD WDSS,1382_01_WDSS_General_Information,https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf,pdf,258,1382,2019-04-25,16.02.2020,1/2-1 OLD NPD WDSS: 1382_01_WDSS_General_Information (pdf)


In [126]:
df_document = df_document[['Wellbore', 'Title', 'Document URL']]
df_document.head(3)

,Wellbore,Title,Document URL
0,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_1 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf
1,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_2 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf
2,1/2-1,1/2-1 OLD NPD WDSS: 1382_01_WDSS_General_Information (pdf),https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf


In [127]:
# Output file
output_to_csv(file='wellbore_document', df=df_document)

Saved to: output data/wellbore_document.csv


,Wellbore,Title,Document URL
0,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_1 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf
1,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_2 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf
2,1/2-1,1/2-1 OLD NPD WDSS: 1382_01_WDSS_General_Information (pdf),https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf


<h3>References and Documents combined</h3>

In [128]:
# Rename columns in df_document
rename_cols = {'Wellbore' : 'Well',
               'Document URL' : 'URL'
               }
    
#Apply renaming
df_document.rename(columns=rename_cols, inplace=True)
df_document.head(3)

,Well,Title,URL
0,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_1 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf
1,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_2 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf
2,1/2-1,1/2-1 OLD NPD WDSS: 1382_01_WDSS_General_Information (pdf),https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf


In [129]:
df_explo_references.head(3)

,Name,Title,URL
3866,1/2-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
1933,1/2-1,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382
3867,1/2-2,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=5192


In [130]:
# Rename columns in df_explo_references
rename_cols = {'Name' : 'Well',
               }
    
#Apply renaming
df_explo_references.rename(columns=rename_cols, inplace=True)
df_explo_references.head(3)

,Well,Title,URL
3866,1/2-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
1933,1/2-1,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382
3867,1/2-2,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=5192


In [131]:
# Combine References and Documents dataframes
df_refs_and_docs = df_explo_references.append(df_document) 
df_refs_and_docs.sort_values(['Well', 'Title'], ascending=[True, False], ignore_index=True, inplace=True)
df_refs_and_docs.head(20)

,Well,Title,URL
0,1/2-1,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382
1,1/2-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
2,1/2-1,1/2-1 REPORTED BY LICENSEE: 1382_1_2_1_COMPLETION_REPORT (pdf),https://factpages.npd.no/pbl/wellbore_documents/1382_1_2_1_COMPLETION_REPORT.pdf
3,1/2-1,1/2-1 REPORTED BY LICENSEE: 1382_1_2_1_COMPLETION_LOG (pdf),https://factpages.npd.no/pbl/wellbore_documents/1382_1_2_1_COMPLETION_LOG.pdf
4,1/2-1,1/2-1 PRESSURE PLOT: 1382_Formation_pressure_(Formasjonstrykk) (pdf),https://factpages.npd.no/pbl/wellbore_pressure_plot_pdfs/1382_Formation_pressure_(Formasjonstrykk).pdf
5,1/2-1,1/2-1 OLD NPD WDSS: 1382_02_WDSS_completion_log (pdf),https://factpages.npd.no/pbl/wdss_old/1382_02_WDSS_completion_log.pdf
6,1/2-1,1/2-1 OLD NPD WDSS: 1382_01_WDSS_General_Information (pdf),https://factpages.npd.no/pbl/wdss_old/1382_01_WDSS_General_Information.pdf
7,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_2 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_2.pdf
8,1/2-1,1/2-1 GEOCHEMICAL INFORMATION: 1382_1 (pdf),https://factpages.npd.no/pbl/geochemical_pdfs/1382_1.pdf
9,1/2-2,Press Release URL,https://www.npd.no/fakta/nyheter/Resultat-av-leteboring/2006/1-2-2/


In [132]:
# Output file
output_to_csv(file='wellbore_references_and_documents', df=df_refs_and_docs)

Saved to: output data/wellbore_references_and_documents.csv


,Well,Title,URL
0,1/2-1,FactPage URL,https://factpages.npd.no/factpages/default.aspx?nav1=wellbore&nav2=PageView|Exploration|All&nav3=1382
1,1/2-1,FactMaps URL,https://factmaps.npd.no/factmaps/3_0/?run=WellboreExpByNPDID&scale=100000&NPDID=1382
2,1/2-1,1/2-1 REPORTED BY LICENSEE: 1382_1_2_1_COMPLETION_REPORT (pdf),https://factpages.npd.no/pbl/wellbore_documents/1382_1_2_1_COMPLETION_REPORT.pdf


<h3>Summary</h3>

In [133]:
df_summary = pd.DataFrame({'Data type':
                        ['Exploration well headers',
                         'Development well headers',
                         'Exploration references',
                         'Development references',
                         'Cores',
                         'Core photos',
                         'Thin sections',
                         'CO2',
                         'Oil samples',
                         'Lithostratigraphy',
                         'Drill stem tests',
                         'Casing and leak-off tests',
                         'Drilling mud',
                         'Documents',
                         'Documents & References combined'
                        ], 
                        'No. unique wells':
                        [df_explo['Name'].nunique(),
                         df_dev['Name'].nunique(),
                         df_explo_references['Well'].nunique(),
                         df_dev_references['Name'].nunique(),
                         df_core['Well'].nunique(),
                         df_core_photo['Well'].nunique(),
                         df_thin_section['Well'].nunique(),
                         df_co2['Well'].nunique(),
                         df_oil_sample['Well'].nunique(),
                         df_formation_top['Well'].nunique(),
                         df_dst['Well'].nunique(),
                         df_casinglot['Well'].nunique(),
                         df_mud['Well'].nunique(),
                         df_document['Well'].nunique(),
                         df_refs_and_docs['Well'].nunique()
                        ],
                        'No. records':
                        [df_explo['Name'].shape[0],
                         df_dev['Name'].shape[0],
                         df_explo_references['Well'].shape[0],
                         df_dev_references['Name'].shape[0],
                         df_core['Well'].shape[0],
                         df_core_photo['Well'].shape[0],
                         df_thin_section['Well'].shape[0],
                         df_co2['Well'].shape[0],
                         df_oil_sample['Well'].shape[0],
                         df_formation_top['Well'].shape[0],
                         df_dst['Well'].shape[0],
                         df_casinglot['Well'].shape[0],
                         df_mud['Well'].shape[0],
                         df_document['Well'].shape[0],
                         df_refs_and_docs['Well'].shape[0]]
                       })

print('Data for', (df_explo['Name'].nunique()+df_dev['Name'].nunique()), 'wells in total.')

#Add thousands separators
df_summary['No. unique wells'] = df_summary['No. unique wells'].apply(lambda x : "{:,}".format(x))
df_summary['No. records'] = df_summary['No. records'].apply(lambda x : "{:,}".format(x))
df_summary = df_summary.style.hide_index()
df_summary

Data for 7087 wells in total.


Data type,No. unique wells,No. records
Exploration well headers,"1,933","1,933"
Development well headers,"5,154","5,154"
Exploration references,"1,933","4,598"
Development references,"5,154","10,308"
Cores,"1,691","8,141"
Core photos,814,"20,849"
Thin sections,200,"2,136"
CO2,52,94
Oil samples,598,"1,005"
Lithostratigraphy,"1,777","36,035"


# Part 2: Prepare data for import to SQL Server

In [134]:
# Import additional modules used in this section

from datetime import datetime
from xlrd import xldate
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, delete, insert, select, func, sql
from sqlalchemy.types import Integer
import urllib
import pprint

In [135]:
# Configure and connect to SQL Server database

pp = pprint.PrettyPrinter(indent=10)

params = 'DRIVER={ODBC Driver 13 for SQL Server};' \
         'SERVER=5SQFPQ2\SQLEXPRESS;' \
         'PORT=1433;' \
         'DATABASE=Test17;' \
         'Trusted_Connection=yes;'
            
params = urllib.parse.quote_plus(params)

engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

metadata = MetaData()
  
#Create Table objects
wells = Table('WELLS', metadata, autoload=True, autoload_with=engine)
wellsuserfieldsvalues = Table('t_WellsUserFieldsValues', metadata, autoload=True, autoload_with=engine)
wellsuserfields = Table('t_WellsUserFields', metadata, autoload=True, autoload_with=engine)
datalithostrat = Table('DATA_Lithostrat', metadata, autoload=True, autoload_with=engine)
projects = Table('PROJECTS', metadata, autoload=True, autoload_with=engine)
wellqueries = Table('WELLQUERIES', metadata, autoload=True, autoload_with=engine)
projectwells = Table('PROJECTWELLS', metadata, autoload=True, autoload_with=engine)
tablesnames = Table('tablenames', metadata, autoload=True, autoload_with=engine)
intervalcolumns = Table('INTERVALCOLUMNS', metadata, autoload=True, autoload_with=engine)
datacore = Table('DATA_Core', metadata, autoload=True, autoload_with=engine)
# = Table('', metadata, autoload=True, autoload_with=engine)

connection = engine.connect()
                       
#pp.pprint(repr(wells))

In [136]:
# Create a function to select all rows from table

def sqlselect_rows(tablename):
    
    select_stmt = select(tablename)
    result = connection.execute(select_stmt)
    
    # Equivalent to:
    # result = engine.execute('SELECT * FROM PROJECTS')

    for row in result:
        print(row)
    
    result.close()

<h3>Populate dbo.WELLS</h3>

In [137]:
# All column titles in dbo.WELLS table

# ic_dbowells_columns = {"pk_index", "well_id", "units", "created", "creator", "modified", "modifier", "project", 
#                           "rte", "sea_bed", "rig_elevation", "datum", "terminal_depth", "spud_date", "completion_date", 
#                           "quadrant", "sub_block", "kelly", "symbol_id", "client", "utmzone", "code", "name", 
#                           "field", "location", "country", "basin", "name1", "name2", "strat_schemes", "grnd_elev", 
#                           "f_block", "grid_x", "grid_y", "latitude", "longtitude", "geodatum", "facility", 
#                           "discovery_name", "seismic_line", "intent", "f_ipid", "f_licenceNumber", "f_api", 
#                           "f_comment", "f_province", "f_county", "f_state", "f_section", "f_Township", "f_range", "f_uwi"}


# Rename well header columns to match dbo.WELLS
rename_for_sql = {'Name' : 'name',
                'Alternate 1' : 'name1',
                'Operator' : 'client',
                'Licence number' : 'f_licenceNumber',
                'Intent' : 'intent',
                'Field' : 'field',
                'SPUD date' : 'spud_date',
                'Completion date' : 'completion_date',
                'Discovery name' : 'discovery_name',
                'Seismic line' : 'seismic_line',
                'Country' : 'country',
                'KBE' : 'kelly',
                'Terminal depth' : 'terminal_depth',
                'Water depth' : 'sea_bed',
                'Location' : 'location',
                'Facility' : 'facility',
                'Geodatum' : 'geodatum',
                'Latitude' : 'latitude',
                'Longitude' : 'longtitude', #spelled incorrectly to match column longtitude in dbo.WELLS
                'Grid system' : 'utmzone',
                'Surface X' : 'grid_x',
                'Surface Y' : 'grid_y',
                'Quadrant' : 'quadrant',
                'Block' : 'f_block'}
    
# Apply renaming to each of the dataframes
df_explo.rename(columns=rename_for_sql, inplace=True)
df_dev.rename(columns=rename_for_sql, inplace=True)

# QC renamed columns
print("Renamed attributes only:")
renamed_columns = list(rename_for_sql.values())
df_explo[renamed_columns].head(3)
# df_dev[renamed_columns].head(3)

Renamed attributes only:


,name,name1,client,f_licenceNumber,intent,field,spud_date,completion_date,discovery_name,seismic_line,country,kelly,terminal_depth,sea_bed,location,facility,geodatum,latitude,longtitude,utmzone,grid_x,grid_y,quadrant,f_block
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,BLANE,1989-03-20,1989-06-04,1/2-1 Blane,PW 8303A - 10 SP. 290,NORWAY,22.0,3574.0,72.0,NORTH SEA,ROSS ISLE,ED50,56.887519,2.476583,ED50 / UTM zone 31N,468106.29,6305128.26,1,2
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,None,2005-12-14,2006-02-02,None,inline 7429-trace 4824 Survey PGS CGMNOR,NORWAY,40.0,3434.0,74.0,NORTH SEA,MÆRSK GIANT,ED50,56.992222,2.496572,ED50 / UTM zone 31N,469410.10,6316774.33,1,2
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,None,1968-07-06,1968-11-11,1/3-1,LINE 5651 SP. E165,NORWAY,26.0,4877.0,71.0,NORTH SEA,ORION,ED50,56.855833,2.851389,ED50 / UTM zone 31N,490936.87,6301488.86,1,3


In [138]:
now = datetime.now()
now

timestampStr = now.strftime("%Y-%m-%d %H:%M:%S.%f")
print('Current Timestamp : ', timestampStr)

Current Timestamp :  2020-02-16 15:02:14.345239


In [139]:
# Convert 'now' timestamp to OLE date/variant date

def datetime2ole(date):
    
    #convert date string to a datetime object
    date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f")
    
    #Calculate OLE manually from OLE origin date
    OLE_TIME_ZERO = datetime(1899, 12, 30)
    delta = date - OLE_TIME_ZERO
    
    return float(delta.days) + (float(delta.seconds) / 86400)  # 86,400 seconds in day

now = datetime2ole(timestampStr)
now

43877.626550925925

In [140]:
# Add new columns for dbo.WELLS that are not in well header file

df_explo['datum'] = 4
df_dev['datum'] = 4
df_explo['symbol_id'] = 3146 #correct later, refer to Status?
df_dev['symbol_id'] = 3146 #correct later, refer to Status?
df_explo['units'] = 'M'
df_dev['units'] = 'M'
df_explo['created'] = now
df_dev['created'] = now
df_explo['creator'] = 1 #correct later
df_dev['creator'] = 1 #correct later

# df_explo['modified'] = null
# df_dev['modified'] = null
# df_explo['modifier'] = null
# df_dev['modifier'] = null
# df_explo['project'] = null
# df_dev['project'] = null

# Check the result
df_explo[['name', 'datum', 'symbol_id', 'units', 'created', 'creator']].head(3)

,name,datum,symbol_id,units,created,creator
0,1/2-1,4,3146,M,43877.626551,1
1,1/2-2,4,3146,M,43877.626551,1
2,1/3-1,4,3146,M,43877.626551,1


In [141]:
# Duplicate columns, preserving the original
df_explo['well_id'] = df_explo['NPDID wellbore']
df_dev['well_id'] = df_dev['NPDID wellbore']

df_explo['f_uwi'] = df_explo['NPDID wellbore']
df_dev['f_uwi'] = df_dev['NPDID wellbore']

df_explo['code'] = df_explo['name']
df_dev['code'] = df_dev['name']

# Limit seismic_line to match nvarchar(80) limit
df_explo["seismic_line"] = df_explo["seismic_line"].str[:80]

# Check the result
df_explo[['name', 'well_id', 'f_uwi', 'code', 'seismic_line']].head(n=10)

,name,well_id,f_uwi,code,seismic_line
0,1/2-1,1382,1382,1/2-1,PW 8303A - 10 SP. 290
1,1/2-2,5192,5192,1/2-2,inline 7429-trace 4824 Survey PGS CGMNOR
2,1/3-1,154,154,1/3-1,LINE 5651 SP. E165
3,1/3-2,165,165,1/3-2,LINE 5656 SP.5125
4,1/3-3,87,87,1/3-3,8186 - 216 sp264-8186-407 sp463
5,1/3-4,2,2,1/3-4,8186 - 208 AND 8186 - 421 crossing of lines
6,1/3-5,223,223,1/3-5,79111 SP. 527
7,1/3-6,1521,1521,1/3-6,LINJE SH 8902 - 161 SP436
8,1/3-7,2505,2505,1/3-7,BPN 9202 inline 1084 & xline3062
9,1/3-8,2829,2829,1/3-8,RANDOM LINES SH 9201 S1 OG S2


In [142]:
# Filter and re-order explo_dbwells to match dbo.WELLS (minus those not required, listed below)

explo_dbowells_order = ["well_id", "units", "created", "creator", "modified", "modifier", "project", 
                        "sea_bed", "datum", "terminal_depth", "spud_date", "completion_date", "quadrant", "kelly", 
                        "symbol_id", "client", "utmzone", "code", "name", "field", "location", "country", "name1", 
                        "f_block", "grid_x", "grid_y", "latitude", "longtitude", "geodatum", "facility", "discovery_name", 
                        "seismic_line", "intent", "f_licenceNumber", "f_uwi"]

df_explo_dbowells = df_explo.filter(explo_dbowells_order)

df_explo_dbowells = df_explo_dbowells.reindex(columns=explo_dbowells_order)

df_explo_dbowells.head(3)


# NOT REQUIRED
#Columns in dbo.WELLS (correct order) that are not in df_explo_dbowells:
#rte, rig_elevation, sub_block, basin, name2, strat_schemes, grnd_elev, "f_ipid, f_api, f_comment, 
#f_province, f_county, f_state, f_section, f_Township, f_range

,well_id,units,created,creator,modified,modifier,project,sea_bed,datum,terminal_depth,spud_date,completion_date,quadrant,kelly,symbol_id,client,utmzone,code,name,field,location,country,name1,f_block,grid_x,grid_y,latitude,longtitude,geodatum,facility,discovery_name,seismic_line,intent,f_licenceNumber,f_uwi
0,1382,M,43877.626551,1,NaN,NaN,NaN,72.0,4,3574.0,1989-03-20,1989-06-04,1,22.0,3146,Phillips Petroleum Norsk AS,ED50 / UTM zone 31N,1/2-1,1/2-1,BLANE,NORTH SEA,NORWAY,1/2-1,2,468106.29,6305128.26,56.887519,2.476583,ED50,ROSS ISLE,1/2-1 Blane,PW 8303A - 10 SP. 290,WILDCAT,143,1382
1,5192,M,43877.626551,1,NaN,NaN,NaN,74.0,4,3434.0,2005-12-14,2006-02-02,1,40.0,3146,Paladin Resources Norge AS,ED50 / UTM zone 31N,1/2-2,1/2-2,None,NORTH SEA,NORWAY,1/2-2,2,469410.10,6316774.33,56.992222,2.496572,ED50,MÆRSK GIANT,None,inline 7429-trace 4824 Survey PGS CGMNOR,WILDCAT,143 CS,5192
2,154,M,43877.626551,1,NaN,NaN,NaN,71.0,4,4877.0,1968-07-06,1968-11-11,1,26.0,3146,A/S Norske Shell,ED50 / UTM zone 31N,1/3-1,1/3-1,None,NORTH SEA,NORWAY,1/3-1,3,490936.87,6301488.86,56.855833,2.851389,ED50,ORION,1/3-1,LINE 5651 SP. E165,WILDCAT,011,154


In [143]:
df_explo_dbowells.tail(3)

,well_id,units,created,creator,modified,modifier,project,sea_bed,datum,terminal_depth,spud_date,completion_date,quadrant,kelly,symbol_id,client,utmzone,code,name,field,location,country,name1,f_block,grid_x,grid_y,latitude,longtitude,geodatum,facility,discovery_name,seismic_line,intent,f_licenceNumber,f_uwi
1930,8211,M,43877.626551,1,NaN,NaN,NaN,447.0,4,1210.0,2017-07-19,2017-08-03,7325,32.0,3146,Statoil Petroleum AS,ED50 / UTM zone 35N,7325/4-1,7325/4-1,None,BARENTS SEA,NORWAY,7325/4-1,4,442761.29,8174015.52,73.649319,25.178261,ED50,SONGA ENABLER,7325/4-1 (Gemini Nord),HFC 11.inline 4920.crossline 14516,WILDCAT,855,8211
1931,8735,M,43877.626551,1,NaN,NaN,NaN,239.0,4,4300.0,2019-05-13,2019-06-15,7335,31.0,3146,Equinor Energy AS,ED50 / UTM zone 37N,7335/3-1,7335/3-1,None,BARENTS SEA,NORWAY,7335/3-1,3,402709.14,8214534.56,73.997183,35.837147,ED50,WEST HERCULES,None,ST14005T15 Inline: 4481. Xline: 8373,WILDCAT,859,8735
1932,8228,M,43877.626551,1,NaN,NaN,NaN,253.0,4,1540.0,2017-08-09,2017-09-01,7435,32.0,3146,Statoil Petroleum AS,ED50 / UTM zone 37N,7435/12-1,7435/12-1,None,BARENTS SEA,NORWAY,7435/12-1,12,402277.88,8222886.96,74.071725,35.808628,ED50,SONGA ENABLER,7435/12-1 (Korpfjell),ST14005T15: Inline 7833. X-Line 7829,WILDCAT,859,8228


In [144]:
df_explo_dbowells.dtypes

well_id                     int64
units                      object
created                   float64
creator                     int64
modified                  float64
modifier                  float64
project                   float64
sea_bed                   float64
datum                       int64
terminal_depth            float64
spud_date          datetime64[ns]
completion_date    datetime64[ns]
quadrant                    int64
kelly                     float64
symbol_id                   int64
client                     object
utmzone                    object
code                       object
name                       object
field                      object
location                   object
country                    object
name1                      object
f_block                     int64
grid_x                    float64
grid_y                    float64
latitude                  float64
longtitude                float64
geodatum                   object
facility      

In [145]:
df_explo_dbowells

,well_id,units,created,creator,modified,modifier,project,sea_bed,datum,terminal_depth,spud_date,completion_date,quadrant,kelly,symbol_id,client,utmzone,code,name,field,location,country,name1,f_block,grid_x,grid_y,latitude,longtitude,geodatum,facility,discovery_name,seismic_line,intent,f_licenceNumber,f_uwi
0,1382,M,43877.626551,1,NaN,NaN,NaN,72.0,4,3574.0,1989-03-20,1989-06-04,1,22.0,3146,Phillips Petroleum Norsk AS,ED50 / UTM zone 31N,1/2-1,1/2-1,BLANE,NORTH SEA,NORWAY,1/2-1,2,468106.29,6305128.26,56.887519,2.476583,ED50,ROSS ISLE,1/2-1 Blane,PW 8303A - 10 SP. 290,WILDCAT,143,1382
1,5192,M,43877.626551,1,NaN,NaN,NaN,74.0,4,3434.0,2005-12-14,2006-02-02,1,40.0,3146,Paladin Resources Norge AS,ED50 / UTM zone 31N,1/2-2,1/2-2,None,NORTH SEA,NORWAY,1/2-2,2,469410.10,6316774.33,56.992222,2.496572,ED50,MÆRSK GIANT,None,inline 7429-trace 4824 Survey PGS CGMNOR,WILDCAT,143 CS,5192
2,154,M,43877.626551,1,NaN,NaN,NaN,71.0,4,4877.0,1968-07-06,1968-11-11,1,26.0,3146,A/S Norske Shell,ED50 / UTM zone 31N,1/3-1,1/3-1,None,NORTH SEA,NORWAY,1/3-1,3,490936.87,6301488.86,56.855833,2.851389,ED50,ORION,1/3-1,LINE 5651 SP. E165,WILDCAT,011,154
3,165,M,43877.626551,1,NaN,NaN,NaN,73.0,4,4297.0,1969-05-14,1969-07-27,1,26.0,3146,A/S Norske Shell,ED50 / UTM zone 31N,1/3-2,1/3-2,None,NORTH SEA,NORWAY,1/3-2,3,484786.28,6310443.20,56.936111,2.750000,ED50,SEDNETH I,None,LINE 5656 SP.5125,WILDCAT,011,165
4,87,M,43877.626551,1,NaN,NaN,NaN,68.0,4,4876.0,1982-08-22,1983-03-24,1,25.0,3146,Elf Petroleum Norge AS,ED50 / UTM zone 31N,1/3-3,1/3-3,TAMBAR,NORTH SEA,NORWAY,1/3-3,3,498885.66,6312226.32,56.952378,2.981681,ED50,BORGSTEN DOLPHIN,1/3-3 Tambar,8186 - 216 sp264-8186-407 sp463,WILDCAT,065,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,1411,M,43877.626551,1,NaN,NaN,NaN,408.0,4,2919.0,1989-06-03,1989-08-19,7324,23.0,3146,Den norske stats oljeselskap a.s,ED50 / UTM zone 35N,7324/10-1,7324/10-1,None,BARENTS SEA,NORWAY,7324/10-1,10,413162.08,8120918.38,73.163736,24.313228,ED50,ROSS RIG (2),None,GLR 3-87-19B SP. 3290,WILDCAT,162,1411
1929,7501,M,43877.626551,1,NaN,NaN,NaN,487.0,4,2865.0,2014-06-23,2014-07-21,7325,40.0,3146,Statoil Petroleum AS,ED50 / UTM zone 35N,7325/1-1,7325/1-1,None,BARENTS SEA,NORWAY,7325/1-1,1,441758.31,8203538.99,73.913528,25.116714,ED50,TRANSOCEAN SPITSBERGEN,7325/1-1 (Atlantis),3D survey MC3D-HFC09-inline 2215 & xline 8697,WILDCAT,615,7501
1930,8211,M,43877.626551,1,NaN,NaN,NaN,447.0,4,1210.0,2017-07-19,2017-08-03,7325,32.0,3146,Statoil Petroleum AS,ED50 / UTM zone 35N,7325/4-1,7325/4-1,None,BARENTS SEA,NORWAY,7325/4-1,4,442761.29,8174015.52,73.649319,25.178261,ED50,SONGA ENABLER,7325/4-1 (Gemini Nord),HFC 11.inline 4920.crossline 14516,WILDCAT,855,8211
1931,8735,M,43877.626551,1,NaN,NaN,NaN,239.0,4,4300.0,2019-05-13,2019-06-15,7335,31.0,3146,Equinor Energy AS,ED50 / UTM zone 37N,7335/3-1,7335/3-1,None,BARENTS SEA,NORWAY,7335/3-1,3,402709.14,8214534.56,73.997183,35.837147,ED50,WEST HERCULES,None,ST14005T15 Inline: 4481. Xline: 8373,WILDCAT,859,8735


In [146]:
# Need to install coordinate zones before writing wells
# Note that dbo.PROJECTWELLS is written when I manually add the wells to my project

In [147]:
# Write df to database
# See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

df_explo_dbowells.to_sql('WELLS', engine, if_exists='append', index = False)

print('dbo.WELLS')
#sqlselect_rows([wells])

# Still to correct datum, symbol_id, creator. I think these might have to read other tables.
# Is well_id ok as npdid_wellbore? Will this cause any problems creating new wells in IC?

dbo.WELLS


In [148]:
# Return dbo.WELLS table
# See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html#pandas.read_sql

sql = '''
SELECT *
FROM dbo.WELLS
'''

pd.read_sql_query(sql, engine)

,pk_index,well_id,units,created,creator,modified,modifier,project,rte,sea_bed,rig_elevation,datum,terminal_depth,spud_date,completion_date,quadrant,sub_block,kelly,symbol_id,client,utmzone,code,name,field,location,country,basin,name1,name2,strat_schemes,grnd_elev,f_block,grid_x,grid_y,latitude,longtitude,geodatum,facility,discovery_name,seismic_line,intent,f_ipid,f_licenceNumber,f_api,f_comment,f_province,f_county,f_state,f_section,f_Township,f_range,f_uwi
0,3865,1382,M,2020-02-18 15:02:14,1,None,None,None,0.0,72.0,0.0,4,3574.0,1989-03-20,1989-06-04,1,None,22.0,3146,Phillips Petroleum Norsk AS,ED50 / UTM zone 31N,1/2-1,1/2-1,BLANE,NORTH SEA,NORWAY,None,1/2-1,None,None,None,2,468106.29,6305128.26,56.887519,2.476583,ED50,ROSS ISLE,1/2-1 Blane,PW 8303A - 10 SP. 290,WILDCAT,None,143,None,None,None,None,None,None,None,None,1382
1,3866,5192,M,2020-02-18 15:02:14,1,None,None,None,0.0,74.0,0.0,4,3434.0,2005-12-14,2006-02-02,1,None,40.0,3146,Paladin Resources Norge AS,ED50 / UTM zone 31N,1/2-2,1/2-2,None,NORTH SEA,NORWAY,None,1/2-2,None,None,None,2,469410.10,6316774.33,56.992222,2.496572,ED50,MÆRSK GIANT,None,inline 7429-trace 4824 Survey PGS CGMNOR,WILDCAT,None,143 CS,None,None,None,None,None,None,None,None,5192
2,3867,154,M,2020-02-18 15:02:14,1,None,None,None,0.0,71.0,0.0,4,4877.0,1968-07-06,1968-11-11,1,None,26.0,3146,A/S Norske Shell,ED50 / UTM zone 31N,1/3-1,1/3-1,None,NORTH SEA,NORWAY,None,1/3-1,None,None,None,3,490936.87,6301488.86,56.855833,2.851389,ED50,ORION,1/3-1,LINE 5651 SP. E165,WILDCAT,None,011,None,None,None,None,None,None,None,None,154
3,3868,165,M,2020-02-18 15:02:14,1,None,None,None,0.0,73.0,0.0,4,4297.0,1969-05-14,1969-07-27,1,None,26.0,3146,A/S Norske Shell,ED50 / UTM zone 31N,1/3-2,1/3-2,None,NORTH SEA,NORWAY,None,1/3-2,None,None,None,3,484786.28,6310443.20,56.936111,2.750000,ED50,SEDNETH I,None,LINE 5656 SP.5125,WILDCAT,None,011,None,None,None,None,None,None,None,None,165
4,3869,87,M,2020-02-18 15:02:14,1,None,None,None,0.0,68.0,0.0,4,4876.0,1982-08-22,1983-03-24,1,None,25.0,3146,Elf Petroleum Norge AS,ED50 / UTM zone 31N,1/3-3,1/3-3,TAMBAR,NORTH SEA,NORWAY,None,1/3-3,None,None,None,3,498885.66,6312226.32,56.952378,2.981681,ED50,BORGSTEN DOLPHIN,1/3-3 Tambar,8186 - 216 sp264-8186-407 sp463,WILDCAT,None,065,None,None,None,None,None,None,None,None,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,5793,1411,M,2020-02-18 15:02:14,1,None,None,None,0.0,408.0,0.0,4,2919.0,1989-06-03,1989-08-19,7324,None,23.0,3146,Den norske stats oljeselskap a.s,ED50 / UTM zone 35N,7324/10-1,7324/10-1,None,BARENTS SEA,NORWAY,None,7324/10-1,None,None,None,10,413162.08,8120918.38,73.163736,24.313228,ED50,ROSS RIG (2),None,GLR 3-87-19B SP. 3290,WILDCAT,None,162,None,None,None,None,None,None,None,None,1411
1929,5794,7501,M,2020-02-18 15:02:14,1,None,None,None,0.0,487.0,0.0,4,2865.0,2014-06-23,2014-07-21,7325,None,40.0,3146,Statoil Petroleum AS,ED50 / UTM zone 35N,7325/1-1,7325/1-1,None,BARENTS SEA,NORWAY,None,7325/1-1,None,None,None,1,441758.31,8203538.99,73.913528,25.116714,ED50,TRANSOCEAN SPITSBERGEN,7325/1-1 (Atlantis),3D survey MC3D-HFC09-inline 2215 & xline 8697,WILDCAT,None,615,None,None,None,None,None,None,None,None,7501
1930,5795,8211,M,2020-02-18 15:02:14,1,None,None,None,0.0,447.0,0.0,4,1210.0,2017-07-19,2017-08-03,7325,None,32.0,3146,Statoil Petroleum AS,ED50 / UTM zone 35N,7325/4-1,7325/4-1,None,BARENTS SEA,NORWAY,None,7325/4-1,None,None,None,4,442761.29,8174015.52,73.649319,25.178261,ED50,SONGA ENABLER,7325/4-1 (Gemini Nord),HFC 11.inline 4920.crossline 14516,WILDCAT,None,855,None,None,None,None,None,None,None,None,8211
1931,5796,8735,M,2020-02-18 15:02:14,1,None,None,None,0.0,239.0,0.0,4,4300.0,2019-05-13,2019-06-15,7335,None,31.0,3146,Equinor Energy AS,ED50 / UTM zone 37N,7335/3-1,7335/3-1,None,BARENTS SEA,NORWAY,None,7335/3-1,None,None,None,3,402709.14,8214

<h3>Populate dbo.t_WellsUserFields</h3>

In [149]:
# Well header columns that are not defaults in IC must be created as Well Attributes

print(len(non_default_attributes), 'non-default well attributes:')
print(non_default_attributes)

55 non-default well attributes:
['1st level with HC age', '1st level with HC formation', '2nd level with HC age', '2nd level with HC formation', '3rd level with HC age', '3rd level with HC formation', 'Bottom hole temperature [°C]', 'Completed year', 'Content - planned', 'DISKOS Well Type', 'DISKOS Wellbore Parent', 'Date all updated', 'Date main level updated', 'Date sync NPD', 'Discovery wellbore', 'Drill permit', 'Drilling days', 'Drilling facility category', 'Drilling facility type', 'Entry year', 'Final vertical depth (TVD) [m RKB]', 'Intent - planned', 'Kick off point [m RKB]', 'Licensing activity awarded in', 'Maximum inclination [°]', 'Multilateral', 'NPDID discovery', 'NPDID drilling facility', 'NPDID field', 'NPDID production facility', 'NPDID production licence drilled in', 'NPDID production licence target', 'NPDID site survey', 'NPDID wellbore', 'NPDID wellbore reclassified from', 'Oldest penetrated age', 'Oldest penetrated formation', 'Plugged and abondon date', 'Plugged d

In [150]:
# Call DataFrame constructor on list of attributes

df_wellsuserfields = pd.DataFrame(non_default_attributes, columns =['f_FieldName'])

# And populate other required columns

df_wellsuserfields['f_FieldID'] = range(1,len(df_wellsuserfields)+1)
df_wellsuserfields['f_IsInputUsed'] = False
df_wellsuserfields['f_InputID'] = 0
df_wellsuserfields['f_Description'] = df_wellsuserfields['f_FieldName']
df_wellsuserfields['f_Origin'] = 0
df_wellsuserfields['f_SortOrder'] = range(1,len(df_wellsuserfields)+1)

df_wellsuserfields

,f_FieldName,f_FieldID,f_IsInputUsed,f_InputID,f_Description,f_Origin,f_SortOrder
0,1st level with HC age,1,False,0,1st level with HC age,0,1
1,1st level with HC formation,2,False,0,1st level with HC formation,0,2
2,2nd level with HC age,3,False,0,2nd level with HC age,0,3
3,2nd level with HC formation,4,False,0,2nd level with HC formation,0,4
4,3rd level with HC age,5,False,0,3rd level with HC age,0,5
5,3rd level with HC formation,6,False,0,3rd level with HC formation,0,6
6,Bottom hole temperature [°C],7,False,0,Bottom hole temperature [°C],0,7
7,Completed year,8,False,0,Completed year,0,8
8,Content - planned,9,False,0,Content - planned,0,9
9,DISKOS Well Type,10,False,0,DISKOS Well Type,0,10


In [151]:
df_wellsuserfields.dtypes

f_FieldName      object
f_FieldID         int32
f_IsInputUsed      bool
f_InputID         int64
f_Description    object
f_Origin          int64
f_SortOrder       int32
dtype: object

In [152]:
# Write df to database
df_wellsuserfields.to_sql('t_WellsUserFields', engine, if_exists='append', index = False)

print('dbo.t_WellsUserFields')
sqlselect_rows([wellsuserfields])

IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'uc_t_WellsUserFields1402080371'. Cannot insert duplicate key in object 'dbo.t_WellsUserFields'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]The statement has been terminated. (3621)") [SQL: 'INSERT INTO [t_WellsUserFields] ([f_FieldName], [f_FieldID], [f_IsInputUsed], [f_InputID], [f_Description], [f_Origin], [f_SortOrder]) VALUES (?, ?, ?, ?, ?, ?, ?)'] [parameters: (('1st level with HC age', 1, False, 0, '1st level with HC age', 0, 1), ('1st level with HC formation', 2, False, 0, '1st level with HC formation', 0, 2), ('2nd level with HC age', 3, False, 0, '2nd level with HC age', 0, 3), ('2nd level with HC formation', 4, False, 0, '2nd level with HC formation', 0, 4), ('3rd level with HC age', 5, False, 0, '3rd level with HC age', 0, 5), ('3rd level with HC format

In [ ]:
# Return dbo.t_WellsUserFields

sql = '''
SELECT *
FROM dbo.t_WellsUserFields
'''

pd.read_sql_query(sql, engine)

In [ ]:
#break me here

<h3>Populate dbo.t_WellsUserFieldsValues</h3>

In [ ]:
# Use df_explo to build a new dataframe of wellsuserfieldsvalues
df_explo_nondefaultattributes = df_explo.filter(non_default_attributes)

# Append the name and well_id columns
df_explo_nondefaultattributes['name'] = df_explo_dbowells['name']
df_explo_nondefaultattributes['well_id'] = df_explo_dbowells['well_id']

df_explo_nondefaultattributes.fillna('', inplace=True)

df_explo_nondefaultattributes

In [ ]:
# Convert ALL columns to object/string, as they're all destined for one column.
# f_StringValue column is dtype('O') in df (i.e. Object) and nvarchar(120) in database.
# No point in attempting to maintain datetime data types, as lost when 'melt' is performed below.

df_explo_nondefaultattributes = df_explo_nondefaultattributes.astype(str)

df_explo_nondefaultattributes = df_explo_nondefaultattributes.replace(to_replace=' 00:00:00', value='', regex=True)

df_explo_nondefaultattributes

In [ ]:
#df_explo_nondefaultattributes.dtypes

In [ ]:
# # Dates in Well Attributes appear as "#2019-10-03 00:00:00.0000000"
# # Reformat without time, but maintain datetime64[ns] data type.

# # df_explo_nondefaultattributes['Date all updated'] = pd.to_datetime(df_explo_nondefaultattributes['Date all updated'].dt.strftime('%Y-%m-%d'))
# df_explo_nondefaultattributes['Date main level updated'] = pd.to_datetime(df_explo_nondefaultattributes['Date main level updated'].dt.strftime('%Y-%m-%d'))
# df_explo_nondefaultattributes['Publication date'] = pd.to_datetime(df_explo_nondefaultattributes['Publication date'].dt.strftime('%Y-%m-%d'))
# df_explo_nondefaultattributes['Release date'] = pd.to_datetime(df_explo_nondefaultattributes['Release date'].dt.strftime('%Y-%m-%d'))

# # Convert 'Date sync NPD' to datetime
# df_explo_nondefaultattributes['Date sync NPD'] = pd.to_datetime(df_explo_nondefaultattributes['Date sync NPD'])

# #df_explo_nondefaultattributes.dtypes
# df_explo_nondefaultattributes[['Date all updated',
#                               'Date main level updated',
#                               'Publication date',
#                               'Release date',
#                               'Date sync NPD']]

# # Note that NaT is a pandas null value, pd.NaT

# #df_explo_nondefaultattributes['f_StringValue'].dtypes
# # Loose datetime data types when melting (below)
# # f_StringValue column is dtype('O') in df (i.e. Object) and nvarchar(120) in database

In [ ]:
# Unpivot DataFrame from wide to long format, appropriate for dbo.t_WellsUserFieldsValues
df_explo_nondefaultattributes = pd.melt(df_explo_nondefaultattributes, id_vars='well_id')

# Rename columns to match t_WellsUserFieldsValues
df_explo_nondefaultattributes.columns = ['f_WellId', 'f_FieldName', 'f_StringValue']

print(df_explo_nondefaultattributes['f_WellId'].nunique())
df_explo_nondefaultattributes.sort_values(by='f_WellId').head(55)

In [ ]:
# Merge df_wellsuserfieldsvalues and df_explo_nondefaultattributes
df_wellsuserfieldsvalues = df_wellsuserfields.merge(df_explo_nondefaultattributes, on='f_FieldName', how='inner')

# Limit df_wellsuserfieldsvalues to three columns in dbo.t_WellsUserFieldsValues
df_wellsuserfieldsvalues = df_wellsuserfieldsvalues[['f_WellId', 'f_FieldID', 'f_StringValue']].sort_values(by=['f_WellId','f_FieldID'])
df_wellsuserfieldsvalues

In [ ]:
df_wellsuserfieldsvalues.columns

In [ ]:
# Write entire df_wellsuserfieldsvalues to SQL Server database

df_wellsuserfieldsvalues.to_sql('t_WellsUserFieldsValues', engine, if_exists='append', index=False)

print('dbo.t_WellsUserFieldsValues')
sqlselect_rows([wellsuserfieldsvalues])

# Not sure why f_StringValue values have trailing spaces. These are not shown in IC.

In [ ]:
# Join WELLS, t_WellsUserFields and t_WellsUserFieldsValues

sql = '''
SELECT ufv.f_WellId, w.name, ufv.f_FieldID, uf.f_FieldName, ufv.f_StringValue
FROM dbo.t_WellsUserFieldsValues AS ufv
JOIN dbo.WELLS AS w
ON w.well_id = ufv.f_WellId
JOIN dbo.t_WellsUserFields AS uf
ON uf.f_FieldId = ufv.f_FieldID
--- WHERE (ufv.f_WellId = 99) AND (ufv.f_FieldID = 37)
ORDER BY w.name
'''
temp_df = pd.read_sql_query(sql, engine)
temp_df.head(56)

In [ ]:
break me here

<h3>Populate dbo.PROJECTS</h3>

In [ ]:
# Populate dbo.PROJECTS with relevant fields to create 4 new dynamic projects:
# ALL WELLS, NORWAY NORTH SEA, NORWAY NORWEGIAN SEA & NORWAY BARENTS SEA

# Note: six not null columns:
    # pk_index int e.g. 1, 
    # project_id int e.g. 113,
    # WellGroupFieldIsUserDefined bit, e.g. 0
    # WellOrderFieldsIsUserDefined bit, e.g. 0
    # f_dynamic bit e.g. 1 (for 
    # f_WellQueryId int (0 for non-dynamic or e.g. 129 for "Exploration Offshore" project with project_id = 21)
    
# IC dialogue requires you to enter a project "Title".

# IC auto-selects:
    # "Type: Static"
    # "display units: metres"
    # "Datum: MSL", "Geodatum: ED50", 
    # "Grid system: ED50 / UTM zone 30N". 
    #  Display tab 
        # "Group wells by: [No Grouping]"
        # "Order wells by: Name".

# Database auto-populates:  
    # "TVD_datum: MSL", 
    # defchronodatatype: 0 (74 for several - probably links to Chronostratigraphy - Age), 
    # deftwtdata: NULL, 
    # defchronointerpid: 0, 
    # WellGroupField: 4,0,5,1, etc, 
    # WellGroupFieldIsUserDefined: 0, 
    # WellOrderField: 0,2, 
    # WellOrderFieldIsUserDefined: 0, 
    # WellPatternTable:0, 
    # WellPatternTableLayerField:0, 
    # WellPatternTablePolygonField:0, 
    # DefaultPRMTemplates:{}, 
    # DefaultSummaryCharts{}, 
    # DefaultWellstickTemplates:{}

In [ ]:
# Rename first project rather than overwrite? Update statement?
# Add code to check for existing project_id so as not to conflict?

In [ ]:
# Create dataframe for dynamic projects

dyprojects_data = {'project_id' : ['3', '4', '5', '6'],
                   'title' : ['NORWAY ALL WELLS', 'NORWAY NORTH SEA', 'NORWAY NORWEGIAN SEA', 'NORWAY BARENTS SEA'],
                   #'client' : ['', '', '', ''],
                   #'jobno' : ['', '', '', ''],
                   #'code' : ['', '', '', ''],
                   #'notes' : ['', '', '', ''],
                   'Units' : 'M',
                   'Map' : ['NULL', 'NULL', 'NULL', 'NULL'],
                   'datum' : ['4230', '4230', '4230', '4230'],
                   'utmzone' : 
                   ['ED50 / UTM zone 31N', 'ED50 / UTM zone 31N', 'ED50 / UTM zone 32N', 'ED50 / UTM zone 34N'],
                   'TVD_datum' : ['MSL', 'MSL', 'MSL', 'MSL'],
                   'OWTranslation' : ['2', '2', '2', '2'],
                   #'f_fieldname' : ['', '', '', ''],
                   'defchronodatatype' : ['0', '0', '0', '0'],
                   #'deftstprops' : ['', '', '', ''],
                   'deftwtdata' : ['NULL', 'NULL', 'NULL', 'NULL'],
                   'deffaultsdatatype' : ['0', '0', '0', '0'],
                   'defchronointerpid' : ['0', '0', '0', '0'],
                   'WellGroupField' : ['0', '0', '0', '0'],
                   'WellGroupFieldIsUserDefined' : ['0', '0', '0', '0'],
                   'WellOrderField' : ['0', '0', '0', '0'],
                   'WellOrderFieldIsUserDefined' : ['0', '0', '0', '0'],
                   'WellPatternTable' : ['NULL', 'NULL', 'NULL', 'NULL'],
                   'WellPatternTableLayerField' : ['NULL', 'NULL', 'NULL', 'NULL'],
                   'WellPatternTablePolygonField' : ['NULL', 'NULL', 'NULL', 'NULL'],
                   #'RPMWellTypeField' : ['', '', '', ''],
                   #'DefaultRPMTemplates' : ['', '', '', ''],
                   'DefaultSummaryCharts' : ['{}', '{}', '{}', '{}'],
                   'DefaultWellstickTemplates' : ['{}', '{}', '{}', '{}'],
                   'f_dynamic' : ['1', '1', '1', '1'],
                   'f_WellQueryId' : ['1', '2', '3', '4']}

# Create temp index, not sent to db
df_dyprojects = pd.DataFrame(dyprojects_data, index = ['project_1', 'project_2', 'project_3', 'project_4'])
df_dyprojects.head(3)

In [ ]:
# # Before 'appending' to dbo.PROJECTS, need to delete existing, default IC project.

# delete_stmt = delete(projects)
# result_proxy = connection.execute(delete_stmt)559

# # Print affected row count
# print('Number of rows deleted:', result_proxy.rowcount)

# # Print results of the executing statement to verify there are no rows
# #print('Verify that dbo.PROJECTS is empty:')
# #print(connection.execute(select_stmt).fetchall())

In [ ]:
# Write df to database

df_dyprojects.to_sql('PROJECTS', engine, if_exists='append', index = False)

print('dbo.PROJECTS')
sqlselect_rows([projects])

<h3>Populate dbo.WELLQUERIES</h3>

In [ ]:
# Check: Error creatng new well query after running this

In [ ]:
# Now create well queries to use with dynamic projects

wellqueries_data = {'invertresults' : ['0', '0', '0', '0'], 
                    'category' : ['ProjectFolder(1)', 'ProjectFolder(2)', 'ProjectFolder(3)', 'ProjectFolder(4)'], 
                    'query_id' : ['1', '2', '3', '4'], 
                    'project_id' : ['-1', '-1', '-1', '-1'], 
                    'title' : ['Country = NORWAY', 'Location = North Sea', 'Location = Norwegian Sea', 'Location = Barents Sea'], 
                    'nentries' : ['1', '1', '1', '1'], 
                    'pencolour' : ['0', '0', '0', '0'], 
                    'enttype' : ['4', '4', '4', '4'], 
                    'entdatatype' : ['0', '0', '0', '0'], 
                    'entfunction' : ['=', '=', '=', '='], 
                    'entvalue' : ['NORWAY', 'NORTH SEA', 'NORWEGIAN SEA', 'BARENTS SEA'], 
                    'entinfokey' : ['Country', 'Location', 'Location', 'Location'], 
                    'highlightstyle' : ['1', '1', '1', '1'], 
                    'highlightsymbol' : ['4198', '4198', '4198', '4198']}

# Only pk_index is not null

# Create temp index, not sent to db
df_wellqueries = pd.DataFrame(wellqueries_data, index = ['wellquery_1', 'wellquery_2', 'wellquery_3', 'wellquery_4'])
df_wellqueries.head(3)

In [ ]:
# Write df to database

df_wellqueries.to_sql('WELLQUERIES', engine, if_exists='append', index = False)

print('dbo.WELLQUERIES')
sqlselect_rows([wellqueries])

<h3>Populate dbo.PROJECTWELLS (which wells in which Projects)</h3>

In [ ]:
# Look at first row of wells and project dataframes
# Merge on project = project_id?

df_explo_dbowells.head(1)

In [ ]:
df_dyprojects.head(1)

In [ ]:
# Need to populate the project column in dbo.WELLS
# This appears empty just now because none of my wells have projects.
# Create a function that populates a project number based on well name

def location_projectid(row):
    if row == 'NORTH SEA':
        return 2
    elif row == 'NORWEGIAN SEA':
        return 3
    elif row == 'BARENTS SEA':
        return 4
    else:
        0

df_explo_dbowells['project'] = df_explo_dbowells['location'].apply(location_projectid)
df_explo_dbowells.head(3)

In [ ]:
# In the SQL database, project in WELLS is nvarchar(25) while project_id in PROJECTS is int.
# In the dataframes I'm writing to the database, these 2 columns are float64 and object respectively.
# Works if we convert both to string (or object, the Python equivalent of nvarchar)

df_explo_dbowells['project'] = df_explo_dbowells['project'].astype(str)
df_dyprojects['project_id'] = df_dyprojects['project_id'].astype(str)

print("df_explo_dbowells['project']: ", df_explo_dbowells['project'].dtypes)
print("df_dyprojects['project_id']: ", df_dyprojects['project_id'].dtypes)

In [ ]:
# Need to join dbo.WELLS and dbo.PROJECTS

# dbo.PROJECTWELLS requires 3 columns (not null):
    #pk_index 11 PK int not null
    #well_id FK 2 int not null
    #project_id 2 FK int not null

df_projectwells = df_explo_dbowells.merge(df_dyprojects, left_on='project', right_on='project_id', how='inner')

df_projectwells[['name', 'well_id', 'project', 'project_id']]

In [ ]:
# Filter to just two columns in dbo.PROJECTWELLS
df_projectwells = df_projectwells[['well_id', 'project_id']]

In [ ]:
# Check
df_projectwells[df_projectwells.project_id == '3']

In [ ]:
df_projectwells.to_sql('PROJECTWELLS', engine, if_exists='append', index = False)

print('dbo.PROJECTWELLS')
sqlselect_rows([projectwells])

<h3>Populate dbo.DATA_Lithostrat</h3>

In [ ]:
df_lithostrat.head(3)

In [ ]:
df_dbodatalithostrat = df_lithostrat.copy(deep=True)

In [ ]:
sql = " SELECT * FROM dbo.tablenames WHERE TABLENAME = 'DATA_Lithostrat' "

In [ ]:
sql = " SELECT * FROM dbo.INTERVALCOLUMNS WHERE dest_table = 100 "

pd.read_sql_query(sql, engine)

In [ ]:
# dbo.DATA_Lithostrat stores Gp, Fm and Mbrs.
# Use Level column to create data_type column, numering Group (110), Formation (111) and Members ().

def level_datatypes(row):
    if row == 'GROUP':
        return 110
    elif row == 'FORMATION':
        return 112
    elif row == 'MEMBER':
        return 113
    else:
        0

# Create new column 'data_type' by applying function to 'Level' column
df_dbodatalithostrat['data_type'] = df_dbodatalithostrat['Level'].apply(level_datatypes)
df_dbodatalithostrat.head(10)

In [ ]:
# Rename columns to match dbo.DATA_Lithostrat

rename_stratcols = {'Top depth' : 'top_depth',
                    'Base depth' : 'base_depth',
                    'Legend' : 'legend',
                    'NPDID wellbore' : 'well_id'}

# Apply renaming to dataframe
df_dbodatalithostrat.rename(columns=rename_stratcols, inplace=True)
df_dbodatalithostrat.head(3)

In [ ]:
df_dbodatalithostrat = df_dbodatalithostrat[['well_id', 'data_type', 'top_depth', 'base_depth', 'legend']]
df_dbodatalithostrat.head(3)

In [ ]:
# Insert new columns

df_dbodatalithostrat['symbol_id'] = 0
df_dbodatalithostrat['f_interpid'] = 0
df_dbodatalithostrat['creator'] = 1
df_dbodatalithostrat['modifier'] = 1
df_dbodatalithostrat['source'] = 'Python script'
df_dbodatalithostrat['attr'] = 'test' 
#'{"ZoneColour":-1,"ZoneColourIsIpAuto":true,"EventSymbolId":0,"IsLocked":false,"OriginalZoneIndex":0}'
df_dbodatalithostrat['top_boundary'] = 0
df_dbodatalithostrat['base_boundary'] = 0
df_dbodatalithostrat['created '] = now
df_dbodatalithostrat['modified'] = now
df_dbodatalithostrat['obsno'] = 0
df_dbodatalithostrat['mindepth'] = 0
df_dbodatalithostrat['maxdepth'] = 0
df_dbodatalithostrat['dipangle'] = 0
df_dbodatalithostrat['dipazimuth'] = 0
df_dbodatalithostrat['age'] = 0

# Additional columns in dbo.Lithostrat that don't need autopopulated
# top_age, base_age, owconf, owqual, owkind, owbaseconf, owbasequal, owbasekind, abr, interpreter, remark, geofeature

In [ ]:
# Why is 'created' column entirely NaN (count 35963 rows) after reindexing?

strat_order = ['well_id', 'data_type', 'top_depth', 'symbol_id', 'f_interpid', 'creator', 
              'modifier', 'source', 'attr', 'top_boundary', 'base_depth', 'base_boundary', 'legend', 
              'created', 'modified', 'obsno', 'mindepth', 'maxdepth', 'dipangle', 'dipazimuth', 'age']

df_dbodatalithostrat = df_dbodatalithostrat.reindex(columns=strat_order)
df_dbodatalithostrat.head(3)

# Will 'pk_index' be auto generated again? On this list it came after symbol_id.

In [ ]:
# Where is "extended geological info" stored (Geologic Feature, Remark)
# Note that source column in DATA_Lithostrat is nvarchar(255)  - change seismic_line from 64 to 255?

#df_dbodatalithostrat['created'] = df_dbodatalithostrat['created'].apply(sqlalchemy.DateTime)
#df_dbodatalithostrat['modified'] = pd.to_datetime

#df_dbodatalithostrat[['created', 'modified']] = df_dbodatalithostrat[['created', 'modified']].apply(pd.to_datetime)
df_dbodatalithostrat.dtypes

In [ ]:
# well_id and data_type columns should be int, not float
df_dbodatalithostrat.well_id = df_dbodatalithostrat.well_id.astype('int64')
df_dbodatalithostrat.data_type = df_dbodatalithostrat.data_type.astype('int64')

# When trying to insert 'created' and 'modified' columns to database, get error:
# TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]
# Why are these floats in dbo.DATA_Lithostrat but in other tables like dbo.WELLS they're nvarchar/srt?
# Try importing vs manually adding created and modified fields in IC.
# See what it does with floats!

# Useful doc: Data type mappings between Python and SQL Server
# https://docs.microsoft.com/en-us/sql/advanced-analytics/python/python-libraries-and-data-types?view=sql-server-ver15

# Need to Convert datetime64[ns] column for 'created' and 'modified' to float64
# But you can't do that! TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]

####df_dbodatalithostrat['created'] = df_dbodatalithostrat['created'].astype('float64')
####df_dbodatalithostrat['modified'] = df_dbodatalithostrat['modified'].astype('float64')

#df_dbodatalithostrat.dtypes

In [ ]:
# Temporarily drop 'created' and 'modified' columns
df_dbodatalithostrat.drop(['created', 'modified'], axis=1, inplace=True)
df_dbodatalithostrat.head(3)

In [ ]:
df_dbodatalithostrat[df_dbodatalithostrat['data_type'] == 113]

In [ ]:
# Write df to database

df_dbodatalithostrat.to_sql('DATA_Lithostrat', engine, if_exists='append', index = False)

print('dbo.DATA_Lithostrat')
sqlselect_rows([datalithostrat])

# Rename datatables (NPD)

In [ ]:
# Rename descriptions and titles in dbo.INTERVALCOLUMNS

sql = '''
UPDATE INTERVALCOLUMNS SET description = 'NPD Core Interval', title = 'NPD Core Interval' WHERE data_type = 5280;
UPDATE INTERVALCOLUMNS SET description = 'NPD Group', title = 'NPD Group_NPD' WHERE data_type = 110;
UPDATE INTERVALCOLUMNS SET description = 'NPD Formation', title = 'NPD Formation_NPD' WHERE data_type = 112;
UPDATE INTERVALCOLUMNS SET description = 'NPD Member', title = 'NPD Member_NPD' WHERE data_type = 113;

'''
#pd.read_sql_query(sql, engine)
with engine.begin() as conn:
    conn.execute(sql)

# Populate DATA_Core

In [ ]:
# data_type = 5590 (Core Intervals)
# dest_table = 546 (Core - Conventional)
# DATA_Core

In [ ]:
# tablesnames = Table('tablenames', metadata, autoload=True, autoload_with=engine)
# intervalcolumns = Table('INTERVALCOLUMNS', metadata, autoload=True, autoload_with=engine)
# datacore = Table('DATA_Core', metadata, autoload=True, autoload_with=engine)


In [ ]:
df_dbodatacore = df_core.copy(deep=True)
df_dbodatacore.columns

In [ ]:
df_dbodatacore.head()

In [ ]:
# df_dbodatacore = df_dbodatacore[['Core sample - top depth', 'Core sample -  bottom depth', 'Core sample number', 'NPDID wellbore']]
# df_dbodatacore.head()

In [ ]:
df_dbodatacore.dtypes

In [ ]:
df_core.head()

In [ ]:
# Merge with dbo.df_core to get well_id?

# df_dbodatacore2 = df_dbodatacore.merge(df_core, left_on='Well', right_on='Well', how='inner')

# df_dbodatacore2

df_dbodatacore.drop(columns='Well', inplace=True)

In [ ]:
# Rename columns to match dbo.DATA_Core
df_dbodatacore.columns = ['well_id', 'top_depth', 'base_depth', 'legend']
df_dbodatacore.head(3)

In [ ]:
df_dbodatacore['data_type'] = 5590
# df_dbodatacore['top_depth'] = 
# df_dbodatacore['base_depth'] = 
df_dbodatacore['top_boundary'] = 1
df_dbodatacore['base_boundary'] = 1
df_dbodatacore['symbol_id'] = 0
# df_dbodatacore['legend'] = 
# df_dbodatacore['abr'] = 
# df_dbodatacore['attr'] = '{"ZoneColour":-1df_dbodatacore["ZoneColourIsIpAuto":truedf_dbodatacore["EventSymbolId":0df_dbodatacore["IsLocked":falsedf_dbodatacore["OriginalZoneIndex":0}'
# df_dbodatacore['interpreter'] = 
df_dbodatacore['created'] = now
df_dbodatacore['creator'] = 1
df_dbodatacore['modified'] = now
df_dbodatacore['modifier'] = 1
df_dbodatacore['obsno'] = 0
df_dbodatacore['mindepth'] = 0
df_dbodatacore['maxdepth'] = 0
# df_dbodatacore['remark'] = 
# df_dbodatacore['geofeature'] = 
df_dbodatacore['source'] = 'Script'
df_dbodatacore['dipangle'] = 0
df_dbodatacore['dipazimuth'] = 0
df_dbodatacore['age'] = 0
# df_dbodatacore['owconf'] = 
# df_dbodatacore['owqual'] = 
# df_dbodatacore['owkind'] = 
# df_dbodatacore['owbaseconf'] = 
# df_dbodatacore['owbasequal'] = 
# df_dbodatacore['owbasekind'] = 
df_dbodatacore['top_age'] = 0
df_dbodatacore['base_age'] = 0
df_dbodatacore['f_interpid'] = 0
# df_dbodatacore['well_id'] = 

df_dbodatacore

In [ ]:
# Write df to database

df_dbodatacore.to_sql('DATA_Core', engine, if_exists='append', index = False)

print('dbo.DATA_Core')
sqlselect_rows([datacore])

In [ ]:
sql = " SELECT * FROM DATA_Core "
pd.read_sql_query(sql, engine)

<h3>Extra: Download Core Photos to file</h3>

In [ ]:
# See https://stackabuse.com/download-files-with-python/
# Using the request Module

# Would also be useful to create folders for each wellbore

def save_core_photo():
    
    for index, row in df_core_photo_deletedrows.iterrows(): # Using filterered dataframe for speed
        
        url = row['Core photo URL']
        filename = url.split('/')[-1]
        filepath = 'core_photo_jpgs\\'

        print('Beginning file download with requests: ', url)
        r = requests.get(url)
        
        with open('{}/{}'.format(filepath, filename), 'wb') as f:
            f.write(r.content)
        print('Saved to: {}/{}'.format(filepath, filename))

        # Retrieve HTTP meta-data
        print(r.status_code)
        print(r.headers['content-type'])
        print(r.encoding)
        
#save_core_photo()

# Sometimes getting error: 
# SSLError: HTTPSConnectionPool(host='factpages.npd.no', port=443): 
# Max retries exceeded with url: /pbl/core_photo_jpgs/3279_06_2044_2049m.jpg 
# (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

<h3>Extra: Download and unzip NPD Shapefiles</h3>

In [ ]:
# NPD shapefiles at https://www.npd.no/en/about-us/information-services/available-data/map-services/

npd_shapefiles = {
    'AFEX': 'https://factpages.npd.no/downloads/shape/afxAreaCurrent.zip',
    'AFEX_block': 'https://factpages.npd.no/downloads/shape/afxAreaSplitByBlock.zip',
    'Licence': 'https://factpages.npd.no/downloads/shape/prlAreaCurrent.zip',
    'Licence_block': 'https://factpages.npd.no/downloads/shape/prlAreaSplitByBlock.zip',
    'Licencing APA': 'https://factpages.npd.no/downloads/shape/apaAreaGross.zip',
    'Licencing APA_block': 'https://factpages.npd.no/downloads/shape/apaAreaNet.zip',
    'Wellbore': 'https://factpages.npd.no/downloads/shape/wlbPoint.zip',
    #Ignore Wellbore - Fontfile for presentation TTF
    'BAA': 'https://factpages.npd.no/downloads/shape/baaAreaCurrent.zip',
    'BAA_block': 'https://factpages.npd.no/downloads/shape/baaAreaSplitByBlock.zip',
    'Field': 'https://factpages.npd.no/downloads/shape/fldArea.zip',
    'Discovery': 'https://factpages.npd.no/downloads/shape/dscArea.zip',
    'Facility': 'https://factpages.npd.no/downloads/shape/fclPoint.zip',
    'Survey': 'https://factpages.npd.no/downloads/shape/seaArea.zip',
    'TUF': 'https://factpages.npd.no/downloads/shape/pipLine.zip',
    'Block': 'https://factpages.npd.no/downloads/shape/blkArea.zip',
    'Quadrant': 'https://factpages.npd.no/downloads/shape/qadArea.zip',
    'Sub area': 'https://factpages.npd.no/downloads/shape/subArea.zip'
}

for key, value in npd_shapefiles.items(): 
    print(value)

#print(npd_shapefiles.values)

In [ ]:
# https://factpages.npd.no/downloads/shape/afxAreaCurrent.zip
# See https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url

def save_shapefiles():
    
    for key, value in npd_shapefiles.items(): 
        
        filepath = 'shapefiles\\'
        zip_file_url = value

        print('Beginning file download with requests: ', zip_file_url)
        r = requests.get(zip_file_url)

        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(filepath)

        print('Files extracted to: {}'.format(filepath))
        
#save_shapefiles()

# Import shapefile to IC?
# Populate colours, e.g. Fields and Discoveries with OGW colours?

# Error:
# SSLError: HTTPSConnectionPool(host='factpages.npd.no', port=443): 
# Max retries exceeded with url: /downloads/shape/afxAreaCurrent.zip 
# (Caused by SSLError(SSLError("bad handshake: 
# Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

<h3>Misc</h3>

In [ ]:
# Show all the dataframes used in this notebook
%whos DataFrame

In [ ]:
# Look up status ~ symbol_id from dbo.SYMBOLS
# Status symbols correspond to the Well Symbols dictionary, dic_id 32003.
df_wellsymbols = pd.read_sql('SELECT symbol_id, dic_id, description FROM dbo.SYMBOLS WHERE dic_id=32003', engine)
df_wellsymbols[['symbol_id', 'description']]

In [ ]:
# Look user_id in dbo.userdef which is used in creator & modifier?
# Status symbols correspond to the Well Symbols dictionary, dic_id 32003.
df_users = pd.read_sql('SELECT user_id, usrid, name FROM dbo.userdef', engine)
df_users.head(3)

In [ ]:
# Template to create new entry under dbo.INTERVALCOLUMNS (IC data types)
# Which dbo table holds IC data tables?

#'pk_index' = 
,'data_type' =  5590
,'dest_table' =  546
,'description' =  'Core Interval'
,'title' =  'Core Interval'
,'graphic' =  0
,'zonal' =  1
,'boundaries' = 0
,'legend' = 1
,'dic_driven' = 0 
,'col_width' = 15
,'back_colour' = '1;12648447;255'
,'brush_style' = 0
,'hatch_style' = 0
,'int_colour' = 8;12648447;8454143
,'row_height' = 15
,'table_header' = 'Legend'
,'horiz_justify' = 1
,'vert_justify' = 1
,'orientation' = 0
,'fontname' = 'Arial'
,'fontsize' = 8
,'fontweight' = 0
,'fontitalic' = 0
,'fontunderline' = 0
,'fontcolour' = 0
#,'plotwith' = 
,'style' = 'I'
,'maximum' = 100
,'minimum' = 0
,'units' = '_'
,'horiz_grid' = 0
,'chartstyle' = 0
,'plotsymbol' = 0
,'labelpoints' = 0
,'mergeevents' = 0
,'eventalign' = 0
,'sbugs_igdtype' = 0
,'sbugs_igdplotpos' = 0
,'allowoverlap' = 0
,'alttable' = None
,'altfield' = None
,'plotassequence' = 0
,'owinterp' = None
,'ordering' = 0
,'wellcore' = 1 # Why -1 for existing data type?
,'surface' = None
,'f_order' = 1 # Need to pick new order
,'isdepthage' = 0
,'f_style' = None
,'f_coreshiftid' = 0

